In [1]:
# !pip install argparse
# !pip install beautifulsoup4
# !pip install spacy
# !python -m spacy download en_core_web_sm
# pip install pandarallel

# Import packages

In [1]:
import logging
import os
import re
import glob
import gzip
import tarfile
# import spacy
# en_nlp = spacy.load("en_core_web_sm")
import nltk
import numpy as np
import pathlib
import pandas as pd
pd.set_option("max_colwidth", 400)

from argparse import ArgumentParser
from bs4 import BeautifulSoup
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=38)
from collections import Counter

import warnings
warnings.filterwarnings("ignore")
# warnings.filterwarnings("ignore", 'This pattern has match groups')

def get_csv_size(csv_name):
    get_size = os.path.getsize (os.getcwd() + '/'+csv_name)
    mb_size = get_size/(1024*1024)
    mb_size = round(mb_size,1)
    return mb_size

INFO: Pandarallel will run on 38 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
with open('wiki_relevant.txt', 'r') as file:    
    relevant_rules = file.read().split('\n')
relevant_rules = '\\b' + '|\\b'.join(relevant_rules)

if relevant_rules.endswith('|\\b'):
    relevant_rules =relevant_rules[:-3]   

relevant_rules = re.compile(relevant_rules, re.IGNORECASE)

print('relevant_rules:')
print(relevant_rules)

with open('irelevant_keywords.txt', 'r') as file:    
    irelevant_rules = file.read().split('\n')
irelevant_rules = '\\b' + '|\\b'.join(irelevant_rules)

if irelevant_rules.endswith('|\\b'):
    irelevant_rules =irelevant_rules[:-3]   

irelevant_rules = re.compile(irelevant_rules, re.IGNORECASE)

print('\nirelevant_rules:')
print(irelevant_rules)

relevant_rules:
re.compile('\\b\\w+-\\w+ relations|\\babduct|\\babuse|\\bactivist|\\bacts\\b|\\badministration|\\bagainst women|\\bagencies|\\ballegati|\\bambassador|\\banarch|\\barmed|\\barmy|\\barrest|\\bartillery|\\bassassin, re.IGNORECASE)

irelevant_rules:
re.compile('\\b/\\bstyles?\\b/|\\b/books?/|\\b/cars?/|\\b/earth/|\\b/food/|\\b/future/|\\b/health/|\\b/life/|\\b/living/|\\b/members?/|\\b/money/|\\b/packages?/|\\b/pictures?/|\\b/popular/|\\b/radio/|\\b/videos?, re.IGNORECASE)


# Clean

In [3]:
df = pd.DataFrame(columns=['path','source', 'filename'])
df.path = [str(x) for x in pathlib.Path('./').glob('**/*gz')]
df.source = df.path.apply(lambda x: x.split('/')[0])
df.filename = df.path.apply(lambda x: x.split('/')[1])
df['date'] = df.path.apply(lambda x: re.search(r'_(\d\d\d\d\d\d)',x)[1]).values
df['year'] = df.date.apply(lambda x: x[0:4]).values.astype(int)
df['output'] = '423/'+ df['source']+'_' +df['date']+'.csv'
df

,path,source,filename,date,year,output
0,cna_eng/cna_eng_200401.gz,cna_eng,cna_eng_200401.gz,200401,2004,423/cna_eng_200401.csv
1,cna_eng/cna_eng_200511.gz,cna_eng,cna_eng_200511.gz,200511,2005,423/cna_eng_200511.csv
2,cna_eng/cna_eng_200905.gz,cna_eng,cna_eng_200905.gz,200905,2009,423/cna_eng_200905.csv
3,cna_eng/cna_eng_200909.gz,cna_eng,cna_eng_200909.gz,200909,2009,423/cna_eng_200909.csv
4,cna_eng/cna_eng_200904.gz,cna_eng,cna_eng_200904.gz,200904,2009,423/cna_eng_200904.csv
...,...,...,...,...,...,...
1884,nyt_eng/nyt_eng_201005.gz,nyt_eng,nyt_eng_201005.gz,201005,2010,423/nyt_eng_201005.csv
1885,nyt_eng/nyt_eng_201007.gz,nyt_eng,nyt_eng_201007.gz,201007,2010,423/nyt_eng_201007.csv
1886,nyt_eng/nyt_eng_200509.gz,nyt_eng,nyt_eng_200509.gz,200509,2005,423/nyt_eng_200509.csv
1887,nyt_eng/nyt_eng_200911.gz,nyt_eng,nyt_eng_200911.gz,200911,2009,423/nyt_eng_200911.csv


In [4]:
df.source.unique()

array(['cna_eng', 'tar', 'ltw_eng', 'wpb_eng', 'apw_eng', 'afp_eng',
       'xin_eng', 'nyt_eng'], dtype=object)

In [5]:
# df= df[~df.source.isin(['ltw_eng','tar'])].reset_index(drop=True)
df= df[~df.source.isin(['tar'])].reset_index(drop=True)
df

,path,source,filename,date,year,output
0,cna_eng/cna_eng_200401.gz,cna_eng,cna_eng_200401.gz,200401,2004,423/cna_eng_200401.csv
1,cna_eng/cna_eng_200511.gz,cna_eng,cna_eng_200511.gz,200511,2005,423/cna_eng_200511.csv
2,cna_eng/cna_eng_200905.gz,cna_eng,cna_eng_200905.gz,200905,2009,423/cna_eng_200905.csv
3,cna_eng/cna_eng_200909.gz,cna_eng,cna_eng_200909.gz,200909,2009,423/cna_eng_200909.csv
4,cna_eng/cna_eng_200904.gz,cna_eng,cna_eng_200904.gz,200904,2009,423/cna_eng_200904.csv
...,...,...,...,...,...,...
1005,nyt_eng/nyt_eng_201005.gz,nyt_eng,nyt_eng_201005.gz,201005,2010,423/nyt_eng_201005.csv
1006,nyt_eng/nyt_eng_201007.gz,nyt_eng,nyt_eng_201007.gz,201007,2010,423/nyt_eng_201007.csv
1007,nyt_eng/nyt_eng_200509.gz,nyt_eng,nyt_eng_200509.gz,200509,2005,423/nyt_eng_200509.csv
1008,nyt_eng/nyt_eng_200911.gz,nyt_eng,nyt_eng_200911.gz,200911,2009,423/nyt_eng_200911.csv


In [6]:
duplicated_idx = df[(df.source == 'nyt_eng') & (df.year >= 2001)].index
duplicated_idx.shape

(119,)

In [7]:
df = df[~df.index.isin(duplicated_idx)].reset_index(drop=True)
df

,path,source,filename,date,year,output
0,cna_eng/cna_eng_200401.gz,cna_eng,cna_eng_200401.gz,200401,2004,423/cna_eng_200401.csv
1,cna_eng/cna_eng_200511.gz,cna_eng,cna_eng_200511.gz,200511,2005,423/cna_eng_200511.csv
2,cna_eng/cna_eng_200905.gz,cna_eng,cna_eng_200905.gz,200905,2009,423/cna_eng_200905.csv
3,cna_eng/cna_eng_200909.gz,cna_eng,cna_eng_200909.gz,200909,2009,423/cna_eng_200909.csv
4,cna_eng/cna_eng_200904.gz,cna_eng,cna_eng_200904.gz,200904,2009,423/cna_eng_200904.csv
...,...,...,...,...,...,...
886,nyt_eng/nyt_eng_199912.gz,nyt_eng,nyt_eng_199912.gz,199912,1999,423/nyt_eng_199912.csv
887,nyt_eng/nyt_eng_199512.gz,nyt_eng,nyt_eng_199512.gz,199512,1995,423/nyt_eng_199512.csv
888,nyt_eng/nyt_eng_200007.gz,nyt_eng,nyt_eng_200007.gz,200007,2000,423/nyt_eng_200007.csv
889,nyt_eng/nyt_eng_199905.gz,nyt_eng,nyt_eng_199905.gz,199905,1999,423/nyt_eng_199905.csv


In [8]:
def extract_text(file_path, type="story", with_headline = True):
    
    # 1 read the file as html so that we can extract content by its tags
    if file_path.endswith('.gz'):
        with gzip.open(file_path) as file:
            soup = BeautifulSoup(file.read(), "html.parser")
    else:    
        with open(file_path) as file:
            soup = BeautifulSoup(file, "html.parser")

    story_list = soup.find_all('doc',type='story')
    output = []
    for story in story_list:
        id_ = story['id']
        try:
            headline = story.headline.text
            headline = re.sub("^\s+|\s+$", "", headline, flags=re.UNICODE) 
        except:
            headline = None
        find_p = story.find_all('p')

        if len(find_p)==0:
            text = story.text           
        else:
            text=[]
            for p in find_p:
                paragraph = p.text
                paragraph = re.sub(r"\n+", " ", paragraph)
                text.append(paragraph)
            text = '\n'.join(text)
            
        output.append({'id': id_, 'title':headline, 'text':text})
    return output

def clean_text(x):
#     x= re.sub('\n(ad|advertisement|tweet):?\n', "", x, flags=re.IGNORECASE) 
    x=  re.sub(r'\n+', "\n", x, flags=re.IGNORECASE) 
    x = re.sub(r"http\S+", "", x)
    x = re.sub("``|''", "\"", x, flags=re.IGNORECASE) 
    x = re.sub("^\s+|\s+$", "", x, flags=re.UNICODE)
    return x

find_relevant = lambda x: Counter(relevant_rules.findall(x.lower()))
find_exclude = lambda x: Counter(irelevant_rules.findall(x.lower()))

def process(row):

    df1 = extract_text(row['path'])
    df1 = pd.DataFrame(df1)
    df1 = df1.rename(columns={'headline':'title'})
    
    sizes=[]
    sizes.append(df1.shape[0])
    
    df1 = df1[~df1.text.isnull()]
    df1= df1[df1.text.str.len()>150]
    df1.loc[df1.title.isnull(),'title'] ='\n'
    sizes.append(df1.shape[0])
    
    # Remove some noises
    df1.text = df1.text.apply(clean_text)
    
    # Remove duplicated
    df1['date'] = df1['id'].apply(lambda x:re.search(r'_(\d+)\.\d+',x)[1])
    df1['start'] = df1['text'].apply(lambda x:x[0:150])

    df1 = df1[~df1.duplicated(subset='text')]
    df1 = df1[~df1.duplicated(subset=['title', 'start'])]
    sizes.append(df1.shape[0])
    
    
    df1['head_in'] = (df1['title'] + ' ' + df1['start']).apply(find_relevant)
    df1['head_out'] =(df1['title'] +' ' + df1['start']).apply(find_exclude)

    df1['text_in'] = df1['text'].apply(find_relevant)
    df1['text_out'] = df1['text'].apply(find_exclude)
    
    
    df1['head_in_count']= df1['head_in'].str.len()
    df1['head_out_count']= df1['head_out'].str.len()
    
    df1['body_in_count']= df1.text_in.str.len()
    df1['body_out_count'] = df1.text_out.str.len()
    
    for col in ['head_in','head_out', 'text_in', 'text_out']:
        df1[col]= df1[col].apply(dict)  
    df1 = df1.drop(columns ='start')

    print('\n\n%s\t%s'%(row['path'], sizes))
    try:
        print('%s: %s'%(df1.text[0], sizes))
        df1.to_csv(row['output'], index=False)
    except:
        print('----- No text -----')
    return

In [9]:
df.source.unique()

array(['cna_eng', 'ltw_eng', 'wpb_eng', 'apw_eng', 'afp_eng', 'xin_eng',
       'nyt_eng'], dtype=object)

In [10]:
pandarallel.initialize(nb_workers=39, progress_bar=False)
# batch_size = 38

INFO: Pandarallel will run on 39 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [17]:
_ = df.loc[df.source == 'cna_eng'].parallel_apply(process, axis=1) 



cna_eng/cna_eng_200401.gz	[400, 400, 398]
A total of 212 Taiwan toy manufacturers will participate in 2004 Hong Kong Toy Exhibition on Jan. 6-9, an official of the Taiwan Toy Manufacturers Association said here Saturday. 
 The Taiwan exhibitors will use 319 booths to display their new products and provide information concerning the designers and suppliers of Taiwan toys, he added.: [400, 400, 398]


cna_eng/cna_eng_199709.gz	[493, 491, 491]
Panamanian President Ernesto Perez Balladares met three Taiwan business leaders on Monday at the presidential office. 
 Balladares talked for more than half an hour with Jeffrey Koo, chairman of the Chinese National Association of Industry and Commerce; Kao Chin-ye0n, chairman of the Chinese National Federation of Industries; and Wang You-theng, chairman of the General Chamber of Commerce of the Republic of China. The three are members of a trade delegation that is accompanying President Lee Teng-hui of the Republic of China on his current Latin A

 The officials said that they are currently mulling various strategies to cope with the above in order to help the manufacturers in the district administrative centers tide over present difficulties.: [695, 694, 694]


cna_eng/cna_eng_200011.gz	[634, 634, 631]
A Singapore Airlines jumbo jet bound for Los Angeles from Taipei crashed at Chiang Kai-shek International Airport during takeoff at 11:15 p.m. Tuesday night. 
 The passenger jet exploded and hit a China Airlines passenger jet on the runway. Some injured passengers were rushed to a hospital in Taoyuan.: [634, 634, 631]


cna_eng/cna_eng_200106.gz	[692, 675, 673]
Mainland Chinese leaders are "afraid" of coming to the negotiating table to talk with their Taiwan counterparts because of a lack of confidence, Republic of China President Chen Shui-bian said Thursday. 
 Chen, who arrived in Paraguay Thursday for a three-day state visit, told the Spanish-language news agency EFE in an interview that he has long been eager to reopen the ga

 In the interview, Shieh also invited German tourists to come visit Taiwan, pointing out that Taiwan now offers visa-free treatment to German citizens for stays of up to 30 days.: [858, 858, 826]


cna_eng/cna_eng_200901.gz	[1073, 1073, 660]
Health insurance premiums for children and adolescents  under 18 from low-income families will be fully subsidized by the government starting Jan. 1, 2009, Minister of the Interior Liao Liou-yi announced Wednesday. 
 The Ministry of the Interior (MOI) has allocated NT$100 million (US$3.05 million) in its 2009 budget to serve that end in the initial stage, and some 110,000 underprivileged children and teenagers are expected to be covered under the program, Liao said. The program has been expanded from coverage of children under three years old from poor families. 
 The newly modified measure is aimed at guaranteeing proper medical care for such children and teenagers and helping to ease the financial burden on their families at a time when the count

 He said that the cross-strait dispute stems mostly, if not totally, from Beijing's "absurd insistence" that the island is part of the PRC, stressing that "only when Beijing recognizes the reality will enduring stability and peace settle in the region.": [858, 858, 858]


cna_eng/cna_eng_200605.gz	[1043, 1043, 951]
The ruling Democratic Progressive Party (DPP) is contemplating tacking on to its party platform an anti-corruption chapter to boost the party's image, but this idea was getting the cold shoulder from a leading party lawmaker. 
 Tsai Huang-liang, director of the DPP Cultural and Communications Department said Sunday the chapter will incorporate three principles, including a ban on moonlighting by DPP lawmakers, a requirement for DPP public officials to put their assets into trust, and an avoidance of conflict of interests for them. 
 By public officials, Tsai said, it refers to the chiefs of rural and urban township, members of county and city council and other officials abov

 Chinatrust had previously bid for Nan Shan when AIG, amid financial troubles, put the unit up for sale last year.: [1117, 1098, 889]A total of 2.22 million text messages were sent via the Chunghwa Telecom mobile network in the first hour of 2005, the highest-ever hourly record, company officials said Saturday. 
 The figure compared with 1.7 million recorded in the first hour of 2004, according to Chunghwa Telecom chief engineer Shih Mu-piao. 
 A total of 8.54 million text messages were sent via the company's mobile network on New Year's eve Friday, up nearly 30 percent from the 6.31 million registered on the same day the previous year, Shih said. 
 The highest daily volume last year was recorded on the first day of Lunar New Year, which stood at 12 million, followed by Christmas Day, which stood at some 9 million, Shih added.: [795, 795, 795]



cna_eng/cna_eng_200409.gz	[830, 830, 829]
Republic of China President Chen Shui-bian arrived in Panama Tuesday for a two-day state visit. 
 T

...

 Taiwan politicians present at the meeting included Kaohsiung Mayor Frank Hsieh, Tainan County Magistrate Chen Tang-shan and opposition Legislators Lee Ying-yuan and Lee Ching-hsiung.: [790, 790, 790]


cna_eng/cna_eng_200505.gz	[912, 912, 912]
President Chen Shui-bian left Taipei for a trip to three South Pacific states Sunday. 
 The five-day diplomatic tour will take Chen to three of the Republic of China's allies in the region -- the Marshall Islands, Kiribati and Tuvalu via the U.S. territory of Guam, according to an itinerary disclosed by his office. 
 The three-nation tour is mainly aimed at cementing friendship and cooperation with the ROC's diplomatic allies in the South Pacific, according to James Huang, deputy secretary of the Presidential Office. 
 During his visit, Huang said, Chen will sign joint comuniques with his counterparts of the three nations and deliver speeches at the parliaments of the Marshall Islands and Kiribati. 
 At the invitation of Marshall Islands Preside

 For his part, Li said that Beijing is ready to do its utmost to realize a peaceful solution to the Taiwan question. However, he strongly reiterated China's opposition to Taiwan independence, saying that "we will never, ever allow anyone to use any means to separate Taiwan from China.": [849, 849, 849]


cna_eng/cna_eng_200310.gz	[657, 657, 654]
Singapore's Foreign Minister S. Jayakumar said he does not think the disagreement over Iraq Has damaged the United Nations. 
 Addressing the annual U.N. General Assembly, he said this is not an occasion to revisit old debates over whether the war in Iraq was authorized. 
 So Jayakumar said it has underscored the fact that only the Security Council can authorize intervention when the Permanent Members are in agreement. 
 And that all states, big and small, will do what they must to protect their vital interests, he said, adding that it would be sometime before consensus could be reached on the U.N. role in post-war Iraq.: [657, 657, 654]


cna_e

 Since then, bilateral exchanges have improved further, including the signing and implementation of a working holiday project that benefits the youth of both countries, the establishment of a branch office in Sapporo by Taiwan's representative office in Tokyo, and the launch of two-way flights between Taipei's Songshan Airport and Tokyo's Haneda Airport slated for this October, he added.: [1288, 1266, 1068]


cna_eng/cna_eng_200610.gz	[922, 922, 834]
President Chen Shui-bian spelled out three wishes for the ruling Democratic Progressive Party (DPP) on its 20th anniversary Saturday: to win the year-end Kaohsiung and Taipei mayoral races, gain a legislative majority and ensure the party's rule of the country continues after his term expires in 2008. 
 Flanked by Vice President Annette Lu, Premier Su Tseng-chang and DPP Chairman Yu Shyi-kun, the president enumerated these three wishes to more than 300,000 DPP followers who gathered here to celebrate the party's founding. 
 Taking all the 

 CBC officials said on Tuesday that the lottery will be held every month, with the first issue comprising 30 million tickets at NT$100 apiece.: [652, 651, 650]


cna_eng/cna_eng_200702.gz	[702, 702, 646]
The following are important events scheduled to take place Thursday, Feb. 1: 
 08:30, Premier Su Tseng-chang will inspect preparations for a special exhibition marking the 60th anniversary of the 228 Incident. 
 10:30, The Ministry of Foreign Affairs will hold a news breifing. 
 11:10, President Chen Shui-bain will visit an old community on Alishan.: [702, 702, 646]


cna_eng/cna_eng_200602.gz	[796, 796, 774]
THE NEW YORK TIMES:  
 Bush, Pushing Economy, Seeks to Keep America Competitive 
 The president said tonight that "America is addicted to oil" and set a goal of replacing much of the nation's Mideast oil imports. 
 THE WALL STREET JOURNAL :  
 Bush Aims to Cut Energy Imports 
 Bush made lessening reliance on Mideast oil a goal to fight "tyranny" and called for cutting such imports

 Commenting on the sentence, opposition People First Party legislators Lee Hung-chun and Liu Wen-hsiung said the MOTC should immediately revoke Far Eastern's rights to operate the ETC system -- not only because the bidding process was rigged but because the infrared-activated ETC system has a high failure rate and is far below required standards.: [924, 924, 832]


cna_eng/cna_eng_200411.gz	[855, 855, 854]
A Taiwan Solidarity Union (TSU) official said Monday that there shouldn't be any problems in "pan-green camp" cooperation in employing its vote allocation campaign strategy for the year-end legislative elections. 
 TSU Secretary-General Lin Jih-jia said that he has contacted Lee Ying-yuan, deputy secretary-general of the ruling Democratic Progressive Party (DPP), over the matter and that the two of them will discuss using the strategy in five constituencies in the near future. 
 The "pan-green camp" of the DPP has long used the vote allocation strategy with success to raise the numbe

In [11]:
_ =df.loc[df.source == 'ltw_eng'].parallel_apply(process, axis=1) 



ltw_eng/ltw_eng_200904.gz	[3080, 3008, 2985]
"Harper's Island" 
 Thursdays at 10 p.m. on CBS 
 Basic story: A group of mostly 20-something friends and family gathers on a small, secluded island near Seattle to celebrate the nuptials of real estate heiress Trish (Katie Cassidy) and her boyfriend of more modest means, Henry (Christopher Gorham). But before anyone's said "I do," various degrees of decapitation and dismemberment ensue among the island's guests and longtime residents. The murders continue, one per episode, and everyone who's still alive is considered a suspect. "Harper's Island" is billed as a murder mystery but feels just as much like a prime-time CW soap opera -- think "Scream" meets "One Tree Hill." 
 What it faces: The series, which replaces the on-hiatus "Eleventh Hour," is up against fellow newcomer "Southland" on NBC and "Private Practice" on ABC. 
 What you'll love: "Harper's Island" only lasts 13 episodes (get it? 13!), so commitment-phobes will find it manageabl

 "We get calls and letters from consumers who care about this," she said. "The task force is dedicated to doing something to make the apparel industry a better place to work.": [2418, 2414, 2328]


ltw_eng/ltw_eng_199407.gz	[2737, 2732, 2698]
Former District of Columbia mayor Marion Barry said Thursday that the time he served in prison on a drug charge helped make him "uniquely qualified" to be mayor again because he used it to reflect on what had gone wrong in his administration and what he could do better in the future. 
 "I now know what didn't work right," Barry said. "I have a very good idea of what needs to be done in terms of a vision for the city. I have the energy, experience, wisdom, tenacity, courage, insight," he said. 
 Barry, during a lunch with Washington Post editors and reporters, called his recovery from drug and alcohol abuse "remarkable." He said he now has a new lifestyle that includes prayer, a psychoanalyst, a new wife and attendance at Alcoholics Anonymous meeti

 Obama has proclaimed his wish to set U.S.-Latin relations on a new footing; he shook Chavez's hand at a recent hemispheric summit and has refrained from criticizing the crackdown in Venezuela. That seems to have emboldened leftist leaders to press their own agenda at the OAS. The administration has been trying to finesse the issue: Last week it proposed that the suspension be lifted but that Cuba's reinstatement be linked to steps toward democracy. When that proposal failed to gain traction, U.S. diplomats joined a working group trying to hammer out a compromise. But most likely the administration is fighting an unwinnable battle. By signaling that it cares more about "partnership" with Latin American governments than defending democracy and human rights, it has allowed support for those principles to crumble at the very institution founded to defend them.: [2992, 2945, 2919]


ltw_eng/ltw_eng_200411.gz	[3158, 3139, 3040]
The Ukrainian presidential election remained too close to call 

 In an interview this week, she pointed to measures that took effect in January, including two new sanctuaries near the bay's mouth where crabbing is curtailed and requirements that crab pots have two cull rings, or escape hatches, for undersized crabs.: [2574, 2566, 2535]


ltw_eng/ltw_eng_200707.gz	[3018, 2964, 2911]
----- No text -----


ltw_eng/ltw_eng_199805.gz	[2634, 2630, 2583]
tear him down. 
 "And it's obvious, I think, to the American people that this has been a hard, well-financed, vigorous effort over a long period of time by those who could not contest the ideas that I brought to the table ... and certainly can't quarrel with the consequences and the results of my service," the president said. "And, therefore, personal attack seems legitimate." Professing that his work is undeterred by the controversy, Clinton used some of his bluntest language to date to make clear he shares the contempt his allies repeatedly have voiced toward independent counsel Kenneth W. Starr. 
 PICK

 However, Robert L. Shapiro, Simpson's lead defense attorney, said in court this week that he could not imagine that 20 jurors and alternates could not be picked from among the 304.: [2721, 2717, 2682]


ltw_eng/ltw_eng_200705.gz	[3251, 3194, 3112]
The following editorial appeared in Monday's Washington Post: 
 Twenty months after Hurricane Katrina, public housing residents of New Orleans remain scattered across the country. Many want to go home. They should be able to. But returning to the same squalid and dangerous housing projects that were isolated cauldrons of dysfunction and pathology is neither just nor humane. Katrina changed everything in the Crescent City, and its public housing must change, too. That's what the U.S. Department of Housing and Urban Development is trying to make happen. But a lawsuit filed by the Advancement Project, a Washington-based civil rights organization, against federal, state and city officials is holding things up. 
 New Orleans public housing before

...

 Times staff writer Fred Alvarez contributed to this report.: [3349, 3325, 3225]


ltw_eng/ltw_eng_200604.gz	[2967, 2933, 2864]
Jackie McLean, the internationally known jazz alto saxophonist, composer and educator whose life and career had a positive impact on countless youngsters and numerous proteges, died Friday at his home in Hartford after a long illness. He was 74. 
 McLean's jazz reputation was based on his distinctive, brilliant, emotionally moving alto saxophone playing and creative composing.He was ranked among the top players in jazz history as an alto saxophonist and was a star in Japan. 
 In Hartford -- the New York City native's adopted home since 1970 -- McLean became known for his jazz program at the University of Hartford's Hartt School of music. A pioneer in bringing full recognition to jazz as a legitimate academic pursuit, he established jazz studies as a degree program and founded the Jackie McLean Institute of Jazz at the music school. 
 In the early 1970s, McLean

 Times researcher Janet Lundblad in Los Angeles contributed to this report.: [1784, 1759, 1701]


ltw_eng/ltw_eng_200610.gz	[3517, 3480, 3395]
The following editorial appeared in Saturday's Washington Post: 
 To hear it from officials at the Bush White House, disgraced lobbyist Jack Abramoff was someone they scarcely could have picked out of a lineup. "Well, Abramoff is someone who we don't know a lot about," Republican National Committee Chairman Ken Mehlman, who served as White House political director, said back in January after Abramoff's guilty plea. Press secretary Scott McClellan said Abramoff had only been at the White House for some holiday parties and "a few staff-level meetings" -- about what, or with whom, the White House wouldn't say. 
 But a new report by the House Government Reform Committee, based on three years of e-mails and billing records from Abramoff's former law firm, shows Abramoff and his colleagues billing their clients for 485 lobbying contacts with White Hou

 The Bush administration worked patiently and tirelessly to negotiate the new agreement, which will have the effect of removing Iraq from United Nations supervision on Jan. 1. Having all but destroyed his presidency through mismanagement of the war, Mr. Bush can now fairly argue as he leaves office that his successor will inherit an Iraqi mission that has been stabilized both militarily and politically. That's not the same thing as the "victory" Mr. Bush has often spoken of; Iraq could still unravel if its leaders or the Obama administration act unwisely. There is now, however, a workable road map for winding down the U.S. troop presence in the country and for consolidating the new political system. Mr. Obama will receive this framework from the president and the Iraqi government he has spent the last two years campaigning against. Though we don't expect him to say so, Mr. Obama is fortunate that he was wrong, both about the surge and about the capacity of Iraq's leaders.: [2912, 2864,

 But Mr. Obama is right when he says that his entire career is antithetical to the divisiveness of the Rev. Wright's comments. We've found things to cheer and things to criticize about Mr. Obama during this long campaign, but we don't see how anyone could question his commitment to transcending old racial battles and finding common ground. The Rev. Wright doesn't speak for the candidate, and we hope the pastor doesn't become a continuing excuse for political ads built on racial fears.: [3413, 3355, 3257]


ltw_eng/ltw_eng_200608.gz	[3309, 3287, 3198]
A beauty queen accused of murdering her husband. A woman accused of killing her pastor husband. A man convicted of killing his bride on their honeymoon. A newly married man who disappears from a cruise ship. A boot-camp director accused of murdering a teen-ager. A former model and her boyfriend who disappear on a boating trip. An 18-year-old whose partially clothed body was found in a car trunk. 
 These are among the blood-and-guts stories

 Times staff writer Eric Malnic contributed to this report.: [3533, 3501, 3422]


ltw_eng/ltw_eng_200708.gz	[3288, 3222, 3175]
----- No text -----


ltw_eng/ltw_eng_200308.gz	[2560, 2551, 2489]
Cost: $1,499. 
 Weight and screen: 6.4 pounds (7.2 pounds with power adapter), 15-inch LCD (1,024 by 768 pixels). 
 Processor and memory: 1.3 GHz Intel Pentium M processor, 256 MB memory, 32 MB ATI Mobility Radeon 7500 graphics accelerator. 
 Storage: 34.3 GB hard drive, 24x/10x/24x CD-RW/8x DVD-ROM combo drive. 
 Communications: 100-Mbps Ethernet, 802.11b WiFi, v.92 modem. 
 Expansion: One PC Card slot, one four-pin FireWire port, two USB 2.0 ports, one parallel port, one serial port, infrared port, one S-Video port, one VGA monitor port. 
 Support: One-year warranty. One year of 24-hour, toll-free phone support; $35-per-incident fee afterward.: [2560, 2551, 2489]


In [12]:
_ =df.loc[df.source == 'wpb_eng'].parallel_apply(process, axis=1) 



wpb_eng/wpb_eng_201002.gz	[1581, 1580, 1555]
When I picked up my chef's knife the first time, its eight-inch blade seemed as scary as a machete. How could I use something that big without drawing blood? 
 I was obviously a kitchen-knife neophyte. That was before I learned how to hold the chef's knife _ in ways I never thought a knife should be held _ and before I learned the all-important (but frankly awkward) "claw," a way of tucking the fingertips of my other hand safely out of the way of a cutting blade. It was before I learned a lot of other things about my knives, too, things that can make a home cook perform more like a sophisticated chef. 
 My transformation began when I went public with my intimidation. As I wrote last year in a Post Food section piece praising my six-inch steak knives, the chef's knife was part of a wedding-registry gift set that had been gathering dust in a block on my countertop because I hadn't a clue how to use it. Instead, I relied on those steak knives

 Fair enough; this site is Comcast's business to run as it sees fit. I'll just say this: I can only wish my employer were doing so well that it could afford to ignore potential markets.: [2076, 2066, 2032]


wpb_eng/wpb_eng_201012.gz	[1767, 1748, 1684]
As I sit here, mostly still full from last week's turkey-and-trimmings marathon and yet nibbling on the last bit of my sons' Halloween candy stash, it occurs to me that 'tis the season for overindulgence, again. 
 And I can't help but wonder what, exactly, the price of all of this holiday- season overeating is, year in and year out. Personally, it feels like I put on at least 10 pounds between Thanksgiving and New Year's Day, what with the extra sweets, stress and parties full of fattening festive foods and booze-soaked toasts, not to mention the cold weather providing even more disincentive than usual to exercise. 
 So I was happily surprised to discover that, in reality, the average holiday weight gain is just under a pound, at least a

 In a nation of more than 250 million passenger vehicles, half a million new electrics a year would barely dent foreign-oil dependence or carbon emissions. And these modest savings will come at the price of federally supported income redistribution _ toward the top.: [1868, 1866, 1824]


wpb_eng/wpb_eng_201003.gz	[2286, 2283, 2239]
It's not only politicians who spin, art historians do it too. 
 In the new Henry Moore exhibition at Tate Britain in London a valiant attempt is made to remodel the celebrated sculptor's image. The show presents "a Moore whose work is darker, edgier, and more complex than the familiar Moore ... we propose that he presents the body as abject, erotic, vulnerable and visceral," writes the curator Chris Stephens in the catalog. 
 In other words, here's a Henry Moore (1898-1986) who was more like his younger contemporary Francis Bacon. I can see why the Tate is tempted to try this art- historical relaunch. Moore, so long taken for granted as the acceptable repres

 Some Republicans now wish Obama were more like Clinton and those "Kumbaya" moments could be replicated. A reminder: Four years after that 1994 election, 95 percent of House Republicans voted to impeach Clinton, basically for lying about sex.: [1838, 1824, 1766]


wpb_eng/wpb_eng_201007.gz	[1927, 1921, 1874]
About a year before the 1988 presidential election, news broke that Democratic front-runner Gary Hart was having an extramarital affair with a young blond model. Within a week, the senator from Colorado quit the race. 
 The media's intense spotlight focused on the model, Donna Rice, then 29. She says she tried to keep silent. Despite losing her job with a pharmaceutical company, she turned down many well-paying, kiss-and-tell opportunities, including one from Playboy offering her "a blank check" for a Q&A session, she says. 
 Now a 52-year-old wife, stepmother and Internet safety advocate, Donna Rice Hughes recalls those months as a painful time: "It felt like I was drowning in a s

 Garcia has advised companies on employment law and human resources for more than 10 years.: [1854, 1845, 1809]


In [13]:
_ =df.loc[df.source == 'apw_eng'].parallel_apply(process, axis=1) 



apw_eng/apw_eng_200412.gz	[3276, 3243, 3170]
The 2004 Atlantic hurricane season officially ended Tuesday, but many people who took a battering from powerful Hurricane Ivan in Grenada say life is still a far cry from back to normal. 
 When Ivan tore through on Sept. 7, it killed 39 people and damaged or destroyed 90 percent of buildings in Grenada. The island has struggled to rebuild and has been receiving millions of dollars in foreign assistance. 
 Although children have returned to school and shops are open in the capital of St. George's, many people still live under tarpaulin roofs or even in tents. 
 Residents say they are still glued to weather reports even though the hurricane season _ one of the strongest in recent history _ is over. 
 "I have to hear the latest weather report (in the) mornings," said Elizabeth Smith, a 56-year-old market vendor. "I am not taking any chances this time. 
 The Atlantic hurricane season, which began June 1, saw four hurricanes that reached Catego

Stanley Kubrick's last film has left too many wide eyes in Singapore's government.  
 Officials here wants to snip part of an orgy scene in "Eyes Wide Shut" before it releases the dark sexual thriller to the general population.  
 Distributor Warner Bros. has appealed the decision by Singapore's Films and Publications Department, saying it is under contract to show the movie in its entirety -- or not at all.  
 "We feel it would be a real waste if people don't get to see it here," Warner Bros. Singapore spokeswoman Li Sok Heng said Tuesday.  
 Warner Bros. first submitted the film's European version, which contains full frontal nudity. When censors asked for cuts, Warner Bros. appealed and submitted an American version, which digitally masks the nude scenes. Censors haven't yet responded to the U.S. version.  
 Singapore, a city-state known for its strict rules and conservative values, routinely censors or bans films, publications and TV programs.  
 "I don't see any reason why they ca

 There was no immediate indication the Guterres-Suharto meeting will quickly resolve that dispute.: [10965, 10912, 10070]


apw_eng/apw_eng_201004.gz	[11138, 10811, 8446]
The man who gunned down one of the nation's few providers of late-term abortions could be sent to prison Thursday for the rest of his life, but he may have gotten what he wanted all along: It is markedly harder in Kansas to get an abortion. 
 Dr. George Tiller's clinic is closed, leaving the state with no facility where women can have the procedure after the 21st week of pregnancy. An early vow by one of Tiller's contemporaries to fill the gap hasn't materialized, and state lawmakers are moving to enact tough new rules to dissuade other physicians from taking Tiller's place. 
 "The national anti-abortion movement has a tremendous victory here," said Dr. Warren Hern, a longtime friend of Tiller who performs late-term abortions in Colorado. "They accomplished exactly what they wanted, and they continue to accomplish it.

 "What needs to be done?" he said. "I'm not really sure. They feel they want a greater share of the pie, and we want our share. We just have to stay strong and fight for what we believe in.": [12651, 12519, 10927]


apw_eng/apw_eng_199503.gz	[13575, 13401, 10900]
Government troops came under heavy attack and rebel forces claimed they had gained significant ground in the see-saw battle for Bosnia's northwest corner.  
 A convoy of nine U.N. aid vehicles had to be abandoned after being caught in Tuesday's cross fire, but there were no reports of injuries, said Maj. Herve Gourmelon, a U.N. military spokesman.  
 The fighting, some the fiercest in the volatile Bihac enclave this year, subsided late Tuesday, another U.N. spokesman, Lt. Col. Gary Coward, reported.  
 Rebel Muslims, along with Serbs from Bosnia and Croatia, attacked positions held by the Muslim-led government south of Velika Kladusa in northwest Bosnia's Bihac region before dawn Tuesday.  
 An ensuing barrage of mortar and ca

 The Asahi also published an April Fools page last year lampooning Japanese politics.: [14408, 14293, 11762]


apw_eng/apw_eng_200911.gz	[12845, 12510, 9853]
Treasury Secretary Timothy Geithner says the economic recovery "could be a little choppy" and it's going to take a while. 
 Geithner told NBC's "Meet the Press" that bringing back jobs and confidence of investors will be the real test of recovery. He declined to say whether the recession is over, saying economist will figure that out years from now. 
 Geithner said in the interview that will air Sunday that this recovery will be different than in the past because Americans are going to have to save more. He noted that a lot of damage was caused by the crisis but there are encouraging signs. 
 An excerpt of the interview was released Saturday night.: [12845, 12510, 9853]


apw_eng/apw_eng_200503.gz	[11808, 11745, 11345]
The phone line in Dave Meyers' third-floor walkup has been turned off to save cash, and the brown velour sofas, b

...

 MORE: [16861, 16630, 14475]


apw_eng/apw_eng_200608.gz	[10660, 10624, 10276]
Argentina's Paola Suarez upset ninth-seeded Dinara Safina 3-6, 6-3, 6-4 in the first round of the Acura Classic on Monday. 
 Suarez, ranked 55th in the world, bounced back from losing the first set to take control of the match against Safina. One of 10 Russians entered in the event, Safina was ranked 15th. 
 Most of the top seeds begin play on Tuesday in the tournament which features six of the world's top 10 players, including No. 2 Kim Clijsters of Belgium. 
 Spanish qualifier Virginia Ruano Pascual, who's won eight Grand Slam doubles titles with Suarez, surprised Elena Likhovtseva 6-4, 6-3. Ruano Pascual, one of eight qualifiers, beat her Russian counterpart with a steady performance. Ruano Pascual, 32, broke Likhovtseva four times. 
 Jelena Jankovic of Serbia pulled off a mild upset with a 6-1, 6-4 victory over 16th-seeded Maria Kirilenko of Russia. Jankovic dominated in the first set before rebounding f

 Graham Rahal, the 20-year-old son of longtime racing star Bobby Rahal, finished fourth, followed by Danica Patrick, earning her fourth top-five finish in the last five races. IndyCar's glamour girl moved into fourth place in the standings, a career high.: [17861, 17287, 13934]


apw_eng/apw_eng_200508.gz	[10552, 10484, 10147]
Last fall, insurgents overran police stations and Iraqi army bases in this northern rural region, scaring off nearly all 2,000 Iraqi troops and keeping people locked inside their homes at night. 
 Last month only two attacks took place in the region mostly populated by Sunni Arabs and Kurds, according to U.S. commanders in the area. 
 The difference, they say, stems from a new approach of relying on sheiks and mukhtars _ the tribal and local leaders who wield enormous influence among some 75,000 people in hundreds of villages and small towns south of the city of Mosul. 
 "Sheiks are the real power here," said Lt. Col. Bradley Becker, commander of the 2nd Battalio

 The AP news report is protected by copyright, which gives the AP exclusive rights to reproduce or authorize reproduction of these materials. AP subscribers are authorized to use the AP news report, subject to the limitations in their news service agreements. Further use is prohibited.: [23690, 22935, 18138]


apw_eng/apw_eng_200903.gz	[17795, 17242, 14275]
Below is a list of feature stories that The Associated Press plans to move in the coming week. Questions about the stories may be addressed to the North America Desk supervisor in New York at 212-621-1650 (fax 212-621-5449 or e-mail amidesk@ap.org) or to individual bureaus in your country or region. 
 We will update this digest daily, adding new features as available. Feature stories that moved in the previous three days are included at the bottom for editors who may not have seen them. 
 Sunday, March 1 
 Rebuilding Gaza -- world faces daunting challenge 
 BEIT LAHIYA, Gaza Strip -- Seven sewage pits in Gaza should have been replac

 The AP: [17795, 17242, 14275]


apw_eng/apw_eng_200912.gz	[13083, 12659, 9832]
Michael Jackson's stunning death made him the Internet's biggest star this year. 
 The quest to find out what happened to Jackson in his final hours on June 25 and celebrate his legacy elevated the late entertainer to the top of the Internet's search charts for 2009. On Yahoo, it ended singer Britney Spears' four-year reign on top. 
 The annual lists released Tuesday were compiled separately by Google Inc. and Yahoo Inc., which combined handle two-thirds of the world's Internet searches. 
 Jackson also grabbed the top spot on the third-largest U.S. search engine, Microsoft Corp.'s Bing, which announced its rankings Sunday. AOL, which relies on Google's search technology, also had Jackson at No. 1. 
 The lists are meant to provide a reading on our cultural pulse. 
 As usual, people using Yahoo's search engine in 2009 seemed to be most interested in celebrities and other diversions, even against the sobering 

 "We don't feel like we're captive to the choices that are handed to us," she said. "That's very different than previous generations.": [16108, 15767, 12687]


apw_eng/apw_eng_199507.gz	[14729, 14663, 11622]
Cyprus Amax Minerals Co. announced Friday it has signed loan agreements totaling $100 million to develop Russia's Kubaka gold mine, the first Western involvement in Russian gold-mining.  
 The Englewood-based company hopes the mine will yield an estimated 310,000 ounces of gold over the next two years.  
 The agreements were signed with the Washington-based U.S. Overseas Private Investment Corp. for $52.5 million and the London-based European Bank for Reconstruction and Development for $47.5 million, Cyprus Amax said.  
 The company said the rest of the funding for the $180 million project will be from joint venture partners.  
 Cyprus Amax is the American sponsor of the joint venture. Through its subsidiary, Cyprus Magadan Gold Mining Co., it owns half of the company that will ove

In [14]:
_ =df.loc[df.source == 'afp_eng'].parallel_apply(process, axis=1) 



afp_eng/afp_eng_200209.gz	[1690, 1668, 1658]
An election poster for Germany's governing Social Democr climax is still to come ..." 
 If politics were sex, the message seems to say, Chancellor Gerhard Sch"meaer must be re-eleartly to his assured response to the floods that ravaged much of eastern Germany. 
 But the opposition conservative Christian Union (CDU/CSU) alliance centred around Edmund Stoiber, his main rival for the chancellery, remains narrowly in front in one of the most closely fought campaigns of recent years. 
 Most polling bodies say the outcome is within the margin of ein post-war German history not to be re-elected for a second term. 
 Stoiber was due Sunday to rally some 10,000 CDU/CSU workers in the western cityH K("pesseldorf in a meeting billed as the launch of the "hot" phase of a campaign that in reality began weeks ago. 
 An unabashed rightwinger with patrician looks, the 60-year-old Stoiber has focused his campaign on the stuployment is Schroeder's Achilles h

 "People welcome in the New Year by idiotically shooting guns into the air, not realizing that those bullets come down at 120 miles per hour," Miami Police Chief John Timoney said in an appeal for people to keep their firearms indoors.: [6729, 6595, 6463]


afp_eng/afp_eng_199501.gz	[10303, 10270, 10033]
Peruvian shoppers descended on stores, markets and sidewalk vendors' stalls Saturday to buy yellow underwear in hopes it would bring them good luck in 1995. 
 As part of New Year's tradition, Lima residents burn old clothes, eat 12 grapes for dinner and set the table with yellow flowers. 
 Women who want to marry in 1995 purchased underwear that combined yellow with red. "Red is for love and yellow is for good luck," one shopper said. "It's a perfect combination, pure fire.": [10303, 10270, 10033]


afp_eng/afp_eng_199410.gz	[10706, 10658, 10413]
India has indefinitely postponed the departure of its first batch of 300 soldiers to Rwanda for UN peacekeeping due to the plague epidemic in

 In a press conference at the end of the meeting Ogata expressed support for a regional conference of Great Lakes countries, one of the recommendations, but she said Rwanda was against it for the time being.: [15308, 15170, 14603]


afp_eng/afp_eng_200809.gz	[12021, 11817, 10382]
Asian elephants can do mathematics, proving their skill at addition in an experiment with their favourite food, a Japanese researcher said Monday.One elephant was 87 percent correct and the other 69 percent right in months of addition exercises involving single digits, according to Naoko Irie of the University of Tokyo. 
 In one test, researchers dropped three apples into one bucket and five into another bucket and then added two apples to each. 
 Five times out of six, Ashya, a 30-year-old female elephant at Tokyo's Ueno Zoo, chose the bucket with seven apples rather than five although she could not see or feel the inside of the containers. 
 The other elephant, 38-year-old Mito from the western city of Kyoto

 Casey said a US-European Union-Russian negotiating team will visit the region soon to consult with the parties involved.: [16021, 15841, 14146]


afp_eng/afp_eng_200706.gz	[15840, 15591, 15242]
Aborigines are 13 times more likely to be jailed than other Australians and have a lower life expectancy, poorer health and smaller incomes than their fellow citizens, a report released Friday said. 
 The government-commissioned report into disadvantages faced by indigenous people found that the number of Aborigines and Torres Strait Islanders behind bars jumped by 32 percent between 2000 and 2006. 
 They now make up about 23.6 percent of the prison population, up from 20 percent in 2002. 
 "Poverty, unemployment, low levels of educational attainment and lack of access to social services are associated with high crime rates and high levels of imprisonment," it said. 
 The Productivity Commission report found that indigenous people have a life expectancy 17 years lower than other Australians -- 

 He cited the example of Australia flanker George Jones as one England should follow: "George Smith played for Australia when he was 19 because he was always going to be the best flanker in Australia, so you are better off playing him and allowing him to accumulate that experience.": [17243, 16992, 14568]


afp_eng/afp_eng_201007.gz	[17226, 15948, 13915]
A suspect in the Russia spy saga cracked after his arrest, confessing he was a Russian agent and pledging greater loyalty to the Kremlin than to his own son, US prosecutors said Thursday. 
 The suspect, who operated under the false identity of Juan Lazaro, also revealed that his Peruvian born journalist wife, Vicky Pelaez, made several trips to South America to give intelligence to their Russian spymasters. 
 They are among 11 "deep-cover" suspects charged with trying to infiltrate US policymaking circles and report back to Moscow in a Cold War-style spy case that has threatened to upset efforts to "reset" ties between the superpowers.

...

afp_eng/afp_eng_199504.gz	[10399, 10358, 10100]
Hans Blix, head of the International Atomic Energy Agency (IAEA), will visit South Korea next week for an annual international meeting of nuclear industries, press reports said Saturday. 
 While staying here from Thursday to Saturday, the director general of the world nuclear watchdog will also meet with President Kim Young-Sam and Foreign Minster Gong Ro-Myung, national Yonhap News Agency said. 
 The Blix visit comes as a US-led plan to replace North Korea's nuclear reactors with safer ones is in a deadlock over the North's rejection of technology from the major potential financer, South Korea. 
 The North was suspected of using its reactors to create nuclear weapons material and agreed to abandon them in exchange for light- water reactors, which produce less plutonium. 
 A government source quoted by Yonhap said the primary purpose of Blix's visit was to attend the nuclear industries meeting but added, "Blix will probably have talks wit

 Factory production has also dropped from government-mandated power rationing, amid a crippling national energy crisis.: [20319, 18458, 15916]


afp_eng/afp_eng_200411.gz	[18276, 18018, 17468]
Factfile for 2006 World Cup and African Nations Cup qualifying competition at halfway stage: 
 Results 
 Preliminary round 
 Niger 0 Algeria 7 (0-1 home, 0-6 away) 
 Botswana 4 Lesotho 1 (4-1 0-0) 
 Swaziland 1 Cape Verde Islands 4 (1-1 0-3) 
 Chad 3 Angola 3 (3-1 0-2, Angola win on away goals) 
 Madagascar 3 Benin 4 (1-1 2-3) 
 Congo 2 Sierra Leone 1 (1-0 1-1) 
 Burundi 1 Gabon 4 (0-0 1-4) 
 Somalia 0 Ghana 7 (0-5 0-2. Both games played in Ghana for security reasons) 
 Guinea 5 Mozambique 3 (1-0 4-3) 
 Tanzania 0 Kenya 3 (0-0 0-3) 
 Gambia 2 Liberia 3 (2-0 0-3) 
 Sao Tome e Principe 0 Libya 9 (0-1 0-8) 
 Ethiopia 1 Malawi 3 (1-3 0-0) 
 Guinea Bissau 1 Mali 4 (1-2 0-2) 
 Rwanda 4 Namibia 1 (3-0 1-1) 
 Sudan 3 Eritrea 0 (3-0 0-0) 
 Equatorial Guinea 1 Togo 2 (1-0 0-2) 
 Uganda 4 Mauritius 3 (3-0 1

 Antonovs are sold mostly to ex-Soviet republics, as well as Middle Eastern and North African countries because they are cheaper than western aircraft.: [9632, 9545, 9275]


afp_eng/afp_eng_200410.gz	[18185, 17971, 17333]
US and Iraqi forces poured into central Samarra overnight and seized the troublespot's government building and police stations in an attempt to beat back the insurgency, the military said in a statement early Friday.: [18185, 17971, 17333]


afp_eng/afp_eng_200204.gz	[14490, 14226, 14061]
Malaysia's trade surplus dropped 13.6 percent in February from the month before, mainly due to the decline in exports of electrical and electronic products, government statistics showed Monday. 
 The country registered a surplus of 3.8 billion ringgit (one billion dollars) in February from 4.4 billion in January, marking the 52nd consecutive monthly trade surplus since November 1997, the International Trade and Industry Ministry said in a statement. 
 The US, along with Singapore, Ja

 In 2007, Venezuela's government did not renew the license of private national television network RCTV, a tough anti-government critic. The government has threatened another network, Globovision, with a shutdown.: [17365, 16248, 14001]


afp_eng/afp_eng_199610.gz	[19044, 18902, 18036]
International rating agency Moody's Investors Service has again drawn the wrath of the Bank of China on Tuesday for assigning a medium grade rating to its two billion Hong Kong dollar (260 million US) floating- rate certificate of deposit. 
 In a statement, the Bank of China, China's main foreign exchange bank, said the "Baa1" rating assigned by Moody's Investors Service to its Hong Kong dollar-denominated floating-rate certificates of deposit issue was "unfair" and "not objective." 
 Moody's said on Monday that the rating reflected "the importance of the bank's business franchises in China and Hong Kong, as well as its weak financial condition, with low profitability and poor asset quality which undermin

 The majority of the 240,000 foreign domestic helpers are from the Philippines with Indonesia and Thailand also important sources.: [17979, 17809, 17320]


afp_eng/afp_eng_200506.gz	[19032, 18840, 18320]
The United States and European Union  must adopt a more flexible attitude if progress is to be made in  their dispute with China over textile exports, state media said  Wednesday ahead of a visit by US trade officials. 
 The leading China Daily, often used by the government to convey  its opinion, said much was at stake and concessions must be made. 
 "As in most trade disputes, unilateral action is not  constructive. Trade wars benefit no one," the newspaper said in an  editorial. 
 "The United States and EU should lessen their demands to keep  the ball rolling." 
 The comments came ahead of a visit to Beijing Thursday by US  Commerce Secretary Carlos Gutierrez for talks with his counterpart  Bo Xilai. 
 US Trade Representative Rob Portman arrives Saturday for  discussions with Vice P

In [15]:
_ =df.loc[df.source == 'xin_eng'].parallel_apply(process, axis=1) 



xin_eng/xin_eng_200604.gz	[4915, 4812, 4797]
A Brazilian plane on a  domestic flight with 19 people on board has disappeared from radar screens Friday night, several hours after it was due to land in  Rio de Janeiro, the country's Civil Aviation Administration said. 
 A search is under way for the aircraft, its 17 passengers and  two crew members, it said. 
 The small double-propeller plane went missing 20 minutes into  the short domestic flight from Macae to Rio de Janeiro. Macae is  180 km east of Rio.: [4915, 4812, 4797]


xin_eng/xin_eng_200101.gz	[6586, 6537, 6476]
At least three explosions from a car bomb rocked Israeli coastal resort town of Netanya Monday evening, injuring 25 people, one seriously.  
 The explosions took place at a bus stop at the junction of Herzl Street and Dizengoff Street in central Netanya, a city 30 kilometers north of Tel Aviv.  
 The injured, including a young man in serious condition, were taken to Beilinson hospital in Petah Tikva near Tel Aviv. Pol

 The army spokesman said that 201 Indian personnel had been killed and 384 others wounded while nine were missing in action.: [7327, 7274, 6948]


xin_eng/xin_eng_200207.gz	[7045, 6949, 6904]
Chinese President Jiang Zemin Monday attended the celebrations for the fifth anniversary of Hong Kong's return to the motherland and the inauguration of the second- term government of the Hong Kong Special Administrative Region ( SAR) at the Hong Kong Convention and Exhibition Center. 
 At 8:57 a.m., accompanied by Chief Executive Tung Chee Hwa and his wife Betty Tung, and amid a burst of enthusiastic applause, Jiang and his wife Wang Yeping entered the Grand Hall, where the ceremony of China's resumption of exercise of sovereignty over Hong Kong was held in 1997. 
 At 9:00 a.m., a 50-member white-uniformed Hong Kong police band played the national anthem to kick off the celebrations. 
 The five-star red flag of the nation and the regional flag of the Hong Kong SAR were hung above the rostrum. Hun

 "We are still trying to work on the matter and by first week of May we can confirm," he added.: [7606, 7552, 7503]


xin_eng/xin_eng_199803.gz	[7718, 7685, 7533]
The United Arab Emirates (UAE) said Saturday that it backs an emergency Opec meeting if "conditions" to ensure the success of the meeting are available.  
 The first of the conditions should be the compliance of all Opec members with their output quotas, not a cut of Opec production ceiling, said Minister of Petroleum and Mineral Resources Obeid bin Saif Al Nasir.  
 "I welcome holding an emergency Opec meeting if all the members are ready to cooperate for its success," he told the UAE's official Wam news agency.  
 The stop of quota excess is the "basic entry" for a successful Opec meeting, he added.  
 Nasir's remarks came as a response to Opec President and Indonesia's Energy Minister Ida Bagus Sujana appeal on Friday for an emergency Opec meeting to discuss output reduction to rescue sagging oil prices caused by ample sup

 Paul Pierce and Erick Strickland added 14 points apiece for Boston who once made the NBA playoff history by coming back from a 21-point deficit in fourth quarter to win Game 3.: [8568, 8054, 7989]


xin_eng/xin_eng_200501.gz	[7878, 7729, 7685]
The destruction in Sri Lanka caused by the Tsunami disaster will be manageable with foreign aid and  that it would not have a major impact on the country's economy,  the Daily News said on Saturday. 
 Deputy Governor Central Bank W. A. Wijewardena speaking at the  media conference on the release of third quarter GDP estimates  2004, he said though the exact damage has not yet been assessed,  the picture is not so gloomy. 
 The World Bank has already pledged 250 million US dollars to  the region, a substantial amount which would come to Sri Lanka.  Other international donor agencies are also surveying the damage  in order to give financial assistance for various needs. 
 Director Statistics Central Bank Anila Dias Bandaranaike said  that although

 In July 2005, China abandoned the yuan peg to dollar, and  raised the value of the RMB by 2 percent to 8.11 per dollar. The  RMB exchange rate had fluctuated around 8 yuan per dollar over the past year, with the rate for July 31, 2006 at 7.9792 yuan against  one dollar.   (more): [8964, 8842, 8822]


xin_eng/xin_eng_200605.gz	[8776, 8617, 8591]
Some 100,000 Argentines  marched on Sunday to an international bridge that links Argentina  with Uruguay, protesting against two Uruguay paper mills for the  prospect of endangering Argentina's environment, Argentine  frontier police said. 
 The protesters ended the march at the San Martin International  Bridge, which links the Argentine city of Gualeguaychu, 230 km  from Argentine capital Buenos Aires, with the Uruguay city of Fray Bentos, where the two paper mills are being built. 
 The protestors said the paper mills will dump waste into the  Uruguay River, which demarcate the border between the two nations, and imperil the environment in Ar

...

 What's more important, she said, people across the world are becoming more cautious with their bio-safety awareness enhanced, as indicated by the fact that the use of genetically engineered seeds was down by 25 percent last year.: [7614, 7553, 7514]


xin_eng/xin_eng_199509.gz	[5993, 5991, 5839]
Unhappy with showing in last month's Iowa straw poll, Republican presidential hopeful Bob Dole is replacing his state campaign manager with a conservative activist from Des Moines, Iowa.  
 "The straw poll was a wake-up call for Dole's supporters, who thought we were so far ahead they didn't need to bother to vote," said Darrell Kearney, who is reportedly replacing professional organizer Steve Gibbs as manager of Dole's Iowa organization. The latter will become director of field operations in Iowa.  
 Kearney, who helped lead Jack Kemp's caucus campaign in the 1988 presidential race, said, "Our job from here on out is to make it clear this is a horse race."  
 In the straw poll last month in A

 Bush is leading Kerry with by 6-8 percentage points in various polls heading to the debate, which was held less than five weeks until the election. But the debates have the potential to change the dynamics in the campaign.: [7557, 7393, 7351]


xin_eng/xin_eng_200908.gz	[7844, 7735, 7721]
Iran Saturday began trials of the " rioters" who were arrested in the disputed presidential election unrests, the official IRNA news agency reported. 
 IRNA said the trials had begun in a revolutionary court, without mentioning the exact number of the "rioters." Local Fars news agency reported that about 100 suspects were in the dock. 
 IRNA reported Wednesday around 20 "rioters" would be put on trial, saying that "indictment bills for some of those, accused of participating in the street riots after the election, have been issued by the Judiciary." 
 "They are charged with being connected to the terrorist groups, bombing, carrying weapons, raiding the Police, Basij (voluntary militia), universities 

 Some 40,000 people have been killed in conflicts fuelled by the PKK's separatist campaign in Turkey.: [10685, 10447, 10424]


xin_eng/xin_eng_200710.gz	[8301, 8173, 8139]
A local Mexican court jailed four Mexicans for human trafficking and prostitution, the nation's Attorney General's Office (PGR) said on Wednesday. 
 Judges of the court in the second district of the northern state of Nuevo Leon convicted the four Mexicans, Walter Alonso Garza, Ingrid Alonso Garza, Pedro Alonso Coronado and Jose Angel Castro, of serving as procurers. 
 They were also found guilty of belonging to a criminal organization with links in Europe, South America, Asia and the Caribbean. The group's main business is to organize parties where women under 25 offered sex to guests. 
 They were arrested on July 14 at the Casa Blanca Hotel in the Nuevo Leon town of Guadalupe. Police also rescued eight women, including four minors. 
 Under Mexico's laws, pimps face prison sentences ranging from two to nine years and

 The private firms also have an important role in mobilizing private funds for AIDS prevention, treatment and care.: [6865, 6823, 6775]


xin_eng/xin_eng_200906.gz	[8310, 8177, 8158]
A bomb explosion at a wholesale vegetable market in southern Baghdad early Monday killed four people and wounded 13 others, an Interior Ministry source said. 
 An explosive charge detonated early in the morning at the al- Rasheed wholesale market in Baghdad's southern district of Doura, the source told Xinhua on condition of anonymity. 
 The bomb was hidden in a shop in the busy market, where dozens of merchants and grocers gather to buy and sell vegetables that usually brought in the early morning from countryside, the source said. 
 The blast destroyed the shop and caused damages to several nearby stalls and civilian vehicles, while Iraqi security forces sealed off the area to secure the scene, the source added. 
 It was the second such deadly attack that struck al-Rasheed wholesale market in less than a

 "We're now responsible to the international community for any  violations of the covenants. We're also obliged to write a  biennial report on our implementation of the covenants," he said.: [8597, 8456, 8416]


In [16]:
_ =df.loc[df.source == 'nyt_eng'].parallel_apply(process, axis=1) 



nyt_eng/nyt_eng_199412.gz	[7238, 7215, 6334]
The Dodgers have decided not to declare an incumbency at their shortstop position, but will attempt to trade Jose Offerman if he doesn't win the starting job next spring. 
 "He needs to reclaim the job or have a change of scenery," Dodgers executive vice president Fred Claire said this week. "If he isn't going to be our starting shortstop, and he's not serving some other role in our lineup, I think (moving him) would be in his best interest, and ours." 
 Neither Offerman nor his agent, Adam Katz, was available for comment. 
 Offerman, 26, has not been the Dodgers' starting shortstop since June 27, when he was demoted to Class-AAA Albuquerque and replaced by slick-fielding but light-hitting Rafael Bournigal. The Dodgers were 38-36 before the move, 20-20 after it, but seemed refreshed by the decision to no longer coddle the struggling fourth-year infielder. 
 To his credit, Offerman didn't sulk at Albuquerque. He hit .330 and scored 43 runs 

 "If I'd had a little more time, I think I could have done better than third here," Penrith said. "It's a new start for me.": [7925, 7905, 6970]


nyt_eng/nyt_eng_199407.gz	[7593, 7567, 6590]
With the nation's attention riveted again on a Los Angeles courtroom, a knife dealer testified that O.J. Simpson bought a 15-inch knife five weeks before the slashing deaths of his ex-wife and her friend. 
 The German-made Stilleto knife Simpson purchased May 3 had a 6-inch stainless steel blade and a handle made of deer antlers with brass trim, said Allen Wattenberg, owner of Ross Cutlery in downtown Los Angeles. 
 Simpson asked that the knife be sharpened and Wattenberg complied, he said. 
 Wattenberg was the first witness to testify Thursday in a hearing to determine if prosecutors have sufficient evidence to bring the former National Football League great to trial for murder in the June 12 slayings of Nicole Brown Simpson and Ronald Goldman. 
 The disclosure -- and new details about evidence i

 And thanks to him, the Knicks could enjoy another victory.: [8041, 8020, 6974]


nyt_eng/nyt_eng_199410.gz	[8099, 8073, 6961]
Gunmen backing army rule scattered a demonstration of supporters of Haiti's elected president Friday as U.S. soldiers, deployed blocks away in tanks surrounded by barbed wire, did nothing to stop the violence. 
 Rifle and pistol shots rang out, rocks flew, and machetes and clubs flailed as a score of the plainclothes auxiliaries known as attaches swaggered out of their hangout in the Normandie Bar and an office of the paramilitary party known as Fraph to block several thousand marchers marking the third anniversary of the military coup that overthrew President Jean-Bertrand Aristide. 
 Journalists counted at least eight bodies in the streets after the clashes, most of them shot by the attaches. But at least two, identified as attaches, were apparently beaten to death by demonstrators. 
 The marchers, growing hesitant and nervous a little before noon as their ro

 "You think if that pitch was close, and me with my no-name on my back, they're going to give it to me?" the catcher said of the 2-2 pitch he took that he said was low and away. "Yeah, right.": [8563, 8549, 7322]


nyt_eng/nyt_eng_200005.gz	[8124, 8115, 6982]
Isaiah Kacyvenski has a great story to tell. 
 It goes beyond the hyperactive linebacker being the highest-round draft choice ever from Harvard. It goes much deeper than what a storybook ending it would be if the Seahawks' fourth-round pick can become the run-stuffing middle linebacker the team seeks. 
 "It is a wonderful story," coach and general manager Mike Holmgren said Sunday after the final practice of Seattle's three-day minicamp. "It is absolutely wonderful." 
 But where to begin this saga of a kid from Endicott, N.Y., who has overcome being the youngest of five children of an alcoholic father and a mother who died during his senior year in high school? Who settled on Harvard as a third option? Who became a fourth-round dr

 -- CHARLES OAKLEY sank a 3-point shot for the second consecutive game after having not made one since 1989.: [9686, 9663, 8352]


nyt_eng/nyt_eng_200002.gz	[9751, 9734, 7842]
With its bright hopes of challenging for the America's Cup beginning to dim, Paul Cayard's America One syndicate sailed a strong, confident race on Tuesday and scored a wire-to-wire victory over Italy's Prada Challenge in the fifth race of the five-of-nine-race Louis Vuitton Cup finals. 
 Prada still maintains a 3-2 edge in the series to determine Team New Zealand's opponent in the America's Cup. That series will begin on Feb. 19. 
 "This is probably the hardest one to win of the four we've got to win," Cayard said. "Tomorrow will be a good one. We'll get it to 3-3 and then we'll see how the other guys like the pressure." 
 Cayard, in need of a victory after two straight defeats, took the action to his counterpart, Francesco de Angelis. First, Cayard tried unsuccessfully to coerce Prada into a prestart foul and h

...

 Bad baseball.: [16374, 16352, 14191]


nyt_eng/nyt_eng_199712.gz	[17767, 17749, 15673]
LOS ANGELES  Though a good chunk of his life was spent building it, Andy Luna never had seen the Getty Center looking quite so stylish before. 
 For 6 1/2 years, the construction foreman had labored on the new Getty Museum and its five affiliated institutes _ collectively, the world's richest arts complex. 
 When he started on the project, Luna said, the Brentwood construction site was "just a hill," a 110-acre slice of the Santa Monica Mountains bereft of buildings, artworks or identity. 
 This past weekend, Luna and his wife Rosalie were among some 5,000 construction workers and their families invited to tour the sprawling marble-and-metal complex, in what the Getty had billed as a celebration of the workers' skill and sweat equity. For Luna and his colleagues, many of them employees of L.A.-based Dinwiddie Construction Co., it was an occasion of proud camaraderie and quiet contemplation. 
 "I've 

 If convicted, Francolino and Malangone face maximum sentences of 25 years.: [17554, 17536, 15094]


nyt_eng/nyt_eng_199611.gz	[15165, 15143, 11703]
The Bruins host two recruits this weekend, 6-foot-9 Kaspars Kambala and 6-foot-8 Jamahl Mosely. Kambala, of Latvia by way of Mequon (Wis.) Homestead High School, is a strong, smart, solid player more developed than George Zidek as a freshman, but not an elite talent. Mosely, of Vista (Calif.) Rancho Buena Vista, is a hard-working power forward. 
 They'll take Kambala if he commits. 
 Baron Davis and Woodbridge power forward Chris Burgess are scheduled to make official visits to Westwood next weekend. Davis has orally committed; Burgess probably will sign with Brigham Young, Duke or Kentucky. 
 Jam session: The Bruins host their annual Fan Jam Sunday at approximately 2 p.m. at Pauley Pavilion. 
 It's free to the public. 
 Formerly known as Fan Appreciation Day, Fan Jam includes scrimmages by the men's and women's teams, 3-point shooting and

 "We're not out of the woods," Kapp agreed. "The Taiwan issue, for one, remains sensitive.": [16432, 16408, 12801]


nyt_eng/nyt_eng_199508.gz	[14551, 14501, 12176]
In what was only a subplot to the Bret Saberhagen watch at Shea Stadium on Monday night, the Mets captured their third straight victory by beating the Pirates, 4-1. It was a noteworthy accomplishment for the team, which had recently endured a six-game losing streak, and for the rookie pitcher Bill Pulsipher, who recorded his first complete-game victory in the major leagues. 
 With the midnight trade deadline approaching, most of the attention was focused on Saberhagen, who spent the game sitting in the Mets' dugout chewing gum and chatting with teammates before his last-minute trade to the Colorado Rockies. But while Saberhagen was being monitored, the Mets did their best to prove why losing their ace pitcher might not be so catastrophic. Rico Brogna belted a two-run homer and Carl Everett delivered a taste of his much-disc

 "I don't know if it was a mistake," Rios said of the decision not to settle. "A lot of things you have to take a gamble and hope things work out your way. We rolled the dice and we went with the superintendent's recommendation. We were trying to show our support of the superintendent. Where are we now? I don't know. We have to reassess and see what comes of this. It's just unfortunate that the court won't stay out of the lives of the children of the Kansas City Missouri School District.": [9270, 9267, 7680]


nyt_eng/nyt_eng_199509.gz	[12688, 12663, 10517]
A lighthearted seminar designed to help Phoenix municipal employees deal with stress turned on three questions. 
 The first two seemed pointed, yet innocuous enough: "What causes you stress?" and "What do you do to rid it?" But the third made some squirm. 
 "Tell us your sexual fantasies," cooed Steve and Daniel Stenson, the brothers who are the seminar's moderators. 
 The question was supposed to loosen up people and to chase away 

 In the end, though, Celtics fans came away disappointed. So did fans of the NBA.: [15060, 15047, 11785]


In [17]:
df = pd.DataFrame(columns=['path','source', 'filename'])
df.path = [str(x) for x in pathlib.Path('423/').glob('**/*csv')]

df.iloc[:,1:] = df.path.str.replace('423/','').str.split('/', expand=True).values
df['date'] = df.path.apply(lambda x: re.search(r'_(\d\d\d\d\d\d)',x)[1]).values
df['year'] = df.date.apply(lambda x: x[0:4]).values.astype(int)
df.source = df.filename.apply(lambda x: re.search(r'(\w+_\w+)_\d',x)[1]).values
df['size'] = df['path'].parallel_apply(get_csv_size)
df

,path,source,filename,date,year,size
0,423/xin_eng_200402.csv,xin_eng,xin_eng_200402.csv,200402,2004,11.6
1,423/nyt_eng_199904.csv,nyt_eng,nyt_eng_199904.csv,199904,1999,42.9
2,423/apw_eng_200904.csv,apw_eng,apw_eng_200904.csv,200904,2009,37.5
3,423/ltw_eng_200407.csv,ltw_eng,ltw_eng_200407.csv,200407,2004,16.0
4,423/xin_eng_200411.csv,xin_eng,xin_eng_200411.csv,200411,2004,11.3
...,...,...,...,...,...,...
874,423/ltw_eng_199712.csv,ltw_eng,ltw_eng_199712.csv,199712,1997,11.5
875,423/ltw_eng_199804.csv,ltw_eng,ltw_eng_199804.csv,199804,1998,11.1
876,423/cna_eng_200805.csv,cna_eng,cna_eng_200805.csv,200805,2008,2.4
877,423/xin_eng_200303.csv,xin_eng,xin_eng_200303.csv,200303,2003,15.4


# Step 2   Filtering use keywords

In [2]:
pandarallel.initialize()
df = pd.DataFrame(columns=['path','source', 'filename'])
df.path = [str(x) for x in pathlib.Path('423/').glob('**/*csv')]

df.iloc[:,1:] = df.path.str.replace('423/','').str.split('/', expand=True).values
df['date'] = df.path.apply(lambda x: re.search(r'_(\d\d\d\d\d\d)',x)[1]).values
df['year'] = df.date.apply(lambda x: x[0:4]).values.astype(int)
df.source = df.filename.apply(lambda x: re.search(r'(\w+_\w+)_\d',x)[1]).values
df['size'] = df['path'].parallel_apply(get_csv_size)
df

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


,path,source,filename,date,year,size
0,423/xin_eng_200402.csv,xin_eng,xin_eng_200402.csv,200402,2004,11.6
1,423/nyt_eng_199904.csv,nyt_eng,nyt_eng_199904.csv,199904,1999,42.9
2,423/apw_eng_200904.csv,apw_eng,apw_eng_200904.csv,200904,2009,37.5
3,423/ltw_eng_200407.csv,ltw_eng,ltw_eng_200407.csv,200407,2004,16.0
4,423/xin_eng_200411.csv,xin_eng,xin_eng_200411.csv,200411,2004,11.3
...,...,...,...,...,...,...
874,423/ltw_eng_199712.csv,ltw_eng,ltw_eng_199712.csv,199712,1997,11.5
875,423/ltw_eng_199804.csv,ltw_eng,ltw_eng_199804.csv,199804,1998,11.1
876,423/cna_eng_200805.csv,cna_eng,cna_eng_200805.csv,200805,2008,2.4
877,423/xin_eng_200303.csv,xin_eng,xin_eng_200303.csv,200303,2003,15.4


In [3]:
def show_statistic(filename):
    df1 = pd.read_csv(filename, header=[0])
    is_relevant = True  if re.search('allafrica|efe|usnews',filename) else False
          
    results =[  df1['head_in_count'].mean(), df1['head_in_count'].std(),
                df1['head_out_count'].mean(),df1['head_out_count'].std(),
                df1['body_in_count'].mean(), df1['body_in_count'].std(),
                df1['body_out_count'].mean(),df1['body_out_count'].std()]  
    
    results = [i.round(2) for i in results] + [is_relevant]
    return pd.Series(results)


new_cols = ['head_in_mean','head_in_std','head_out_mean','head_out_std',
            'body_in_mean','body_in_std','body_out_mean','body_out_std']


pandarallel.initialize(nb_workers=32, progress_bar=False)
df[new_cols +['relevant']]= df['path'].parallel_apply(show_statistic)
df

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


,path,source,filename,date,year,size,head_in_mean,head_in_std,head_out_mean,head_out_std,body_in_mean,body_in_std,body_out_mean,body_out_std,relevant
0,423/xin_eng_200402.csv,xin_eng,xin_eng_200402.csv,200402,2004,11.6,1.43,1.45,0.60,0.94,5.81,5.10,2.49,2.43,False
1,423/nyt_eng_199904.csv,nyt_eng,nyt_eng_199904.csv,199904,1999,42.9,0.80,1.21,0.60,0.86,9.16,8.98,6.40,4.60,False
2,423/apw_eng_200904.csv,apw_eng,apw_eng_200904.csv,200904,2009,37.5,1.38,1.60,0.75,1.03,7.83,8.18,3.91,3.80,False
3,423/ltw_eng_200407.csv,ltw_eng,ltw_eng_200407.csv,200407,2004,16.0,1.13,1.38,0.45,0.75,11.82,10.64,4.98,4.22,False
4,423/xin_eng_200411.csv,xin_eng,xin_eng_200411.csv,200411,2004,11.3,1.48,1.48,0.64,0.96,6.04,5.32,2.58,2.56,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
874,423/ltw_eng_199712.csv,ltw_eng,ltw_eng_199712.csv,199712,1997,11.5,1.00,1.29,0.61,0.88,10.58,9.03,5.70,4.45,False
875,423/ltw_eng_199804.csv,ltw_eng,ltw_eng_199804.csv,199804,1998,11.1,1.08,1.35,0.53,0.82,10.68,9.15,5.21,4.20,False
876,423/cna_eng_200805.csv,cna_eng,cna_eng_200805.csv,200805,2008,2.4,1.21,1.29,0.70,0.99,6.42,5.25,3.04,2.52,False
877,423/xin_eng_200303.csv,xin_eng,xin_eng_200303.csv,200303,2003,15.4,1.66,1.48,0.53,0.87,6.62,5.58,2.14,2.37,False


In [4]:
df.groupby(by=["source"], dropna=False).mean().round(2)

,year,size,head_in_mean,head_in_std,head_out_mean,head_out_std,body_in_mean,body_in_std,body_out_mean,body_out_std,relevant
source,,,,,,,,,,,
afp_eng,2003.27,29.33,1.53,1.60,0.73,1.00,7.88,7.41,3.13,3.17,False
apw_eng,2002.41,29.40,1.53,1.65,0.71,1.01,8.66,8.11,3.62,3.55,False
cna_eng,2003.97,1.45,1.22,1.23,0.78,1.11,6.08,4.92,3.06,2.44,False
ltw_eng,2002.03,12.43,1.14,1.38,0.50,0.80,11.26,9.80,5.07,4.19,False
nyt_eng,1997.23,43.50,0.69,1.10,0.75,0.97,7.66,7.40,6.82,4.61,False
wpb_eng,2010.00,8.28,1.16,1.37,0.58,0.85,11.12,9.01,5.94,4.20,False
xin_eng,2002.46,11.41,1.26,1.41,0.73,1.00,5.30,4.98,2.75,2.56,False


In [5]:
df1= pd.read_csv('423/ltw_eng_199804.csv', header=0)
df1

,id,title,text,date,head_in,head_out,text_in,text_out,head_in_count,head_out_count,body_in_count,body_out_count
0,LTW_ENG_19980401.0002,Both Sides in VMI Abuse Case Face Charges,"Three freshmen at the Virginia Military Institute who said they were struck with a belt for ""infractions"" such as playing country music or getting a low grade are facing the same punishment as the three seniors who allegedly hit them -- expulsion from the school because they initially denied that the incidents took place. \n All six students have been charged with violating VMI's honor code, w...",19980401,"{'abuse': 1, 'case': 1, 'military': 1}",{'music': 1},"{'military': 1, 'incidents': 1, 'violat': 2, 'court': 7, 'expel': 2, 'victim': 1, 'constitution': 1, 'federal': 1, 'case': 4, 'criminal': 1, 'justice': 1, 'department': 1, 'abuse': 2}","{'music': 1, 'invest': 2}",3,1,13,2
1,LTW_ENG_19980401.0003,Stations to Offer Candidates Campaign Forum (Washn),"The six Post-Newsweek stations will offer free air time to qualified gubernatorial and congressional candidates for the November election, according to president and CEO Bill Ryan. \n The format will consist of five-minute segments assembled into a long-form, commercial-free program. \n ""Campaign '98: Straight Talk From the Candidates,"" Tuesday's announcement said, ""will give the participants ...",19980401,"{'congress': 1, 'election': 1}",{},"{'congress': 1, 'election': 1, 'president': 1, 'vote': 3}",{},2,0,4,0
2,LTW_ENG_19980401.0004,Hypocrisy on Campaign Funds,"The following editorial appeared in Wednesday's Washington Post: \n The House Republican leaders have followed the unfortunate example of their Senate counterparts on campaign-finance reform, only even more clumsily. Their goal was to kill reform but avoid the blame. They may have failed at both. Republicans have spent a year and a half claiming to be indignant about the fund-raising abuses in...",19980401,"{'republican': 1, 'senate': 1}",{},"{'republican': 4, 'senate': 4, 'reform': 8, 'kill': 1, 'abuse': 2, 'parties': 1, 'law': 1, 'vote': 4, 'force': 1, 'democratic': 2, 'corrupt': 1}","{'financ': 1, 'fund': 2}",2,0,11,2
3,LTW_ENG_19980401.0012,RENO AT LARGE: Never Underestimate New York Mayor's Petulance,"It's astonishing what New Yorkers will put up with if only you don't murder them, or at least if you murder them at a sharply reduced rate. \n A histrionic mayor whose overextended ego is in perpetual competition with his overdeveloped tendency to imagine slights, for instance. A man to whom a defiant jaywalker, a speeding motorist, a political critic, a drug lord, a mob fish peddler, an imper...",19980401,"{'mayor': 1, 'murder': 2}",{},"{'murder': 7, 'mayor': 8, 'political': 2, 'revolution': 1, 'army': 1, 'armed': 1, 'police': 1}","{'grammy': 1, 'success': 1}",2,0,7,2
4,LTW_ENG_19980401.0013,Why Helen Hunt Is Keeping the Day Job (Hollywood),"TV, or not TV? \n For Helen Hunt, despite winning a best actress Oscar last week for ""As Good as It Gets,"" it made sense -- and more than 2 billion cents, based on her estimated $1 million-per-episode salary -- not to give up her day job on the NBC sitcom ""Mad About You."" \n For George Clooney, star of ""Batman and Robin"" and ""The Peacemaker,"" continuing to make the rounds on ""ER"" no longer see...",19980401,{},"{'hollywood': 1, 'tv': 2, 'winning': 1, 'oscar': 1}","{'party': 2, 'president': 3, 'referendum': 1, 'nypd': 1, 'court': 1}","{'tv': 9, 'winning': 1, 'oscar': 1, 'films': 5, 'movie': 14, 'financ': 1, 'film': 6, 'commodit': 1, 'market': 1, 'entertainment': 1, 'deals': 1, 'career': 2, 'product': 2, 'expos': 1, 'success': 1, 'business': 1, 'price': 1}",0,4,5,17
...,...,...,...,...,...,...,...,...,...,...,...,...
2527,LTW_ENG_19980430.0141,Loyalty to Clintons Emerges in Hubbell Tapes (Washn),"He may have been in jail, but Webster L. Hubbell still saw himself as a Clinton loyalist. \n Talking to his wife in 1996 while in a federal prison serving out his sentence for embezzling from his fo

In [3]:
def select_by_threshold(filename, output_folder):
    df1 = pd.read_csv(filename, header=[0])
    sizes =[]
    sizes.append(df1.shape[0])

    head_in   = df1['head_in_count']
    head_out  = df1['head_out_count']
    body_in   = df1['body_in_count']
    body_out  = df1['body_out_count']

#     remove_index =\
#         ((head_in ==0) & ( head_out > 0)) |\
#         ((head_in > 0) & ( head_out > 1)) |\
#         ( body_in < 2) |\
#         ((body_in >=2) & (body_in < 2*body_out))

#     remove_index = df1[remove_index== True].index
#     sizes.append(remove_index.shape[0])
    
#     df1 = df1[~df1.index.isin(remove_index)]
#     sizes.append(df1.shape[0])

    select_index = (head_in >= head_out) & (body_in > body_out)
    df1 = df1[select_index]
    sizes.append(df1.shape[0])

    filename = filename.split('/')[1]
    new_filename = output_folder + '/'+ filename
    df1.to_csv(new_filename, index= False)
    print('%s:\t%s'%(new_filename, sizes))

In [4]:
pandarallel.initialize(nb_workers=38, progress_bar=False) 
_ = df['path'].parallel_apply(select_by_threshold, output_folder='423_filter')

INFO: Pandarallel will run on 38 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
423_filter/cna_eng_200103.csv:	[795, 494]
423_filter/cna_eng_200907.csv:	[800, 288]
423_filter/cna_eng_200308.csv:	[794, 484]
423_filter/cna_eng_199904.csv:	[774, 496]
423_filter/cna_eng_200811.csv:	[813, 439]
423_filter/cna_eng_200402.csv:	[598, 387]
423_filter/wpb_eng_201010.csv:	[1752, 980]
423_filter/apw_eng_200412.csv:	[3170, 1977]
423_filter/xin_eng_200009.csv:	[7880, 3321]
423_filter/ltw_eng_200909.csv:	[2787, 1468]
423_filter/xin_eng_200104.csv:	[7486, 3874]
423_filter/cna_eng_200512.csv:	[826, 560]423_filter/cna_eng_199808.csv:	[809, 503]

423_filter/xin_eng_200204.csv:	[7360, 4420]
423_filter/xin_eng_200402.csv:	[7612, 4714]
423_filter/xin_eng_200310.csv:	[7882, 4610]
423_filter/cna_eng_200708.csv:	[811, 467]
423_filter/ltw_eng_200805.csv:	[3257, 1684]
423_filter/xin_eng_200707.csv:	[9347, 5155]423_filter/ltw_eng_199708.csv:	[2213, 128

423_filter/afp_eng_199407.csv:	[9655, 5891]
423_filter/afp_eng_200612.csv:	[14250, 8318]
423_filter/afp_eng_199509.csv:	[11209, 7237]
423_filter/ltw_eng_199703.csv:	[2509, 1466]
423_filter/apw_eng_200202.csv:	[14475, 8283]423_filter/apw_eng_200408.csv:	[8847, 4869]

423_filter/afp_eng_200411.csv:	[17468, 11250]
423_filter/afp_eng_200402.csv:	[17450, 10605]
423_filter/afp_eng_200312.csv:	[16657, 10658]
423_filter/apw_eng_200712.csv:	[12182, 7185]
423_filter/cna_eng_200311.csv:	[519, 314]
423_filter/cna_eng_200902.csv:	[727, 319]
423_filter/cna_eng_200012.csv:	[634, 345]
423_filter/cna_eng_200812.csv:	[823, 375]423_filter/wpb_eng_201009.csv:	[1809, 976]423_filter/xin_eng_199506.csv:	[6212, 3362]


423_filter/xin_eng_200401.csv:	[6352, 3876]
423_filter/xin_eng_199712.csv:	[7231, 3486]
423_filter/xin_eng_200101.csv:	[6476, 3147]
423_filter/cna_eng_199803.csv:	[790, 448]
423_filter/apw_eng_200701.csv:	[18138, 10647]
423_filter/cna_eng_200508.csv:	[863, 566]
423_filter/xin_eng_199607.csv:	[7

423_filter/ltw_eng_199607.csv:	[2554, 1576]423_filter/cna_eng_200703.csv:	[873, 528]423_filter/ltw_eng_200601.csv:	[3108, 1751]423_filter/xin_eng_200708.csv:	[8917, 4586]423_filter/ltw_eng_200311.csv:	[3323, 1967]




423_filter/afp_eng_199602.csv:	[12810, 8200]
423_filter/xin_eng_200810.csv:	[7172, 3470]
423_filter/xin_eng_199909.csv:	[7232, 3636]
423_filter/afp_eng_200508.csv:	[7629, 4368]
423_filter/wpb_eng_201003.csv:	[2239, 1239]
423_filter/nyt_eng_199704.csv:	[14629, 4240]
423_filter/apw_eng_201004.csv:	[8446, 4374]423_filter/afp_eng_200803.csv:	[16092, 9088]

423_filter/xin_eng_199610.csv:	[7023, 3701]
423_filter/ltw_eng_200806.csv:	[2930, 1500]
423_filter/apw_eng_199906.csv:	[12147, 7776]
423_filter/apw_eng_199812.csv:	[12402, 8087]
423_filter/xin_eng_200709.csv:	[8825, 4597]
423_filter/xin_eng_200712.csv:	[9054, 5141]
423_filter/nyt_eng_199506.csv:	[11443, 4440]
423_filter/cna_eng_199910.csv:	[694, 465]
423_filter/ltw_eng_199712.csv:	[2567, 1339]
423_filter/xin_eng_199911.csv:

In [5]:
df = pd.DataFrame(columns=['path','source', 'filename'])
df.path = [str(x) for x in pathlib.Path('423_filter/').glob('**/*csv')]

df.iloc[:,1:] = df.path.str.replace('423_filter/','').str.split('/', expand=True).values
df['date'] = df.path.apply(lambda x: re.search(r'_(\d\d\d\d\d\d)',x)[1]).values
df['year'] = df.date.apply(lambda x: x[0:4]).values.astype(int)
df.source = df.filename.apply(lambda x: re.search(r'(\w+_\w+)_\d',x)[1]).values
df['size'] = df['path'].parallel_apply(get_csv_size)
df

,path,source,filename,date,year,size
0,423_filter/xin_eng_200402.csv,xin_eng,xin_eng_200402.csv,200402,2004,7.6
1,423_filter/nyt_eng_199904.csv,nyt_eng,nyt_eng_199904.csv,199904,1999,18.9
2,423_filter/apw_eng_200904.csv,apw_eng,apw_eng_200904.csv,200904,2009,21.5
3,423_filter/ltw_eng_200407.csv,ltw_eng,ltw_eng_200407.csv,200407,2004,10.4
4,423_filter/xin_eng_200411.csv,xin_eng,xin_eng_200411.csv,200411,2004,7.2
...,...,...,...,...,...,...
874,423_filter/ltw_eng_199712.csv,ltw_eng,ltw_eng_199712.csv,199712,1997,6.3
875,423_filter/ltw_eng_199804.csv,ltw_eng,ltw_eng_199804.csv,199804,1998,6.6
876,423_filter/cna_eng_200805.csv,cna_eng,cna_eng_200805.csv,200805,2008,1.5
877,423_filter/xin_eng_200303.csv,xin_eng,xin_eng_200303.csv,200303,2003,10.9


In [6]:
print('total size:', df['size'].sum())
df.groupby(by=["source"], dropna=False)['size'].sum()

total size: 9666.300000000001


source
afp_eng    2664.9
apw_eng    3380.4
cna_eng     131.2
ltw_eng     950.3
nyt_eng    1273.3
wpb_eng      58.0
xin_eng    1208.2
Name: size, dtype: float64

In [7]:
df1 = pd.read_csv('423_filter/xin_eng_200402.csv', header=0)
df1

,id,title,text,date,head_in,head_out,text_in,text_out,head_in_count,head_out_count,body_in_count,body_out_count
0,XIN_ENG_20040201.0001,Sri Lankan govt says ready for cohabitation with president,"Sri Lankan government has said that it is ready to go ahead with what they called a ""structured co- habitation"" government with President Chandrika Kumaratunga which is likely to bring an end to the current political standoff in the country, The Sunday Times reported. \n Malik Samarawickrama, chairman of the ruling United National Party (UNP) of Prime Minister Ranil Wickremesinghe, said on Sat...",20040201,"{'president': 2, 'govern': 2}",{},"{'govern': 5, 'president': 7, 'political': 4, 'party': 1, 'minister': 2, 'committee': 3, 'council': 1, 'defense': 2, 'ministry': 2, 'ministries': 1, 'parliament': 1, 'election': 1}",{},2,0,12,0
1,XIN_ENG_20040201.0006,URGENT: Some 100 Iranian reformist MPs resign over election dispute,"Some 100 Iranian reformist MPs resigned Sunday to protest the mass disqualification of reformer candidates from the upcoming parliamentary elections, members of the Iranian parliament said.",20040201,"{'reform': 3, 'election': 2, 'dispute': 1, 'protest': 1, 'parliament': 1}",{},"{'reform': 2, 'protest': 1, 'parliament': 2, 'election': 1}",{},5,0,4,0
2,XIN_ENG_20040201.0007,Sri Lanka chief truce monitor assumes duty,"Chief truce monitor of the Nordic truce monitoring team in Sri Lanka supervising the ceasefire between the government and the Tamil Tiger rebels officially assumed his duty on Sunday as the new head of the group, officials said. \n Trond Furuhovde, a retired Norwegian army general, arrived in the island on Saturday morning, the spokesperson for the Sri Lanka Monitoring Mission (SLMM) said. \n ...",20040201,"{'govern': 1, 'rebel': 1}",{},"{'govern': 5, 'rebel': 6, 'officials': 1, 'army': 2, 'liberation': 1, 'armed': 1, 'war': 1, 'security': 1, 'defense': 1, 'minister': 1, 'political': 1}",{},2,0,11,0
3,XIN_ENG_20040201.0011,Karzai congratulates Afghans on Eid festival,"Afghanistan's transitional president Hamid Karzai on Sunday congratulated his people on the Eid-al-Adha festival, hoping peace and prosperity in the post-war country, which is still plagued by remnant of terrorism. \n ""I congratulate to the Afghan people on the occasion of Eid, and I wish all the luckiness and prosperity to our people and to the Islamic world,"" the president told reporters whi...",20040201,{'president': 1},{},"{'president': 5, 'war': 1, 'terror': 1, 'ambassador': 1, 'attack': 2, 'militia': 1, 'kill': 1, 'foreign': 1, 'peacekeep': 1, 'soldier': 1, 'bomb': 1, 'nato': 1, 'security': 1, 'force': 1}",{'dollar': 1},1,0,14,1
4,XIN_ENG_20040201.0014,"Bangladesh, Kuwait to sign defense pact","Bangladesh is going to sign a defense pact with Kuwait soon to provide technical and vocational training to Kuwaiti army. \n The Bangladeshi cabinet approved the Bangladesh-Kuwait Defense Pact on Saturday evening. \n A source from Bangladesh army elaborated the pact to Xinhua on Sunday that the pact will be in force for six years, and is renewable for another six years if the two sides agree. ...",20040201,"{'defense': 2, 'army': 1}",{},"{'defense': 4, 'army': 6, 'force': 2, 'armed': 2, 'forces': 1, 'peacekeep': 1, 'war': 1}","{'performance': 1, 'financ': 1, 'currency': 1}",2,0,7,3
...,...,...,...,...,...,...,...,...,...,...,...,...
4709,XIN_ENG_20040229.0278,US marines arrive in chaotic Haitian capital,"The first contingent of US Marines arrived in the chaotic Haitian capital of Port-au-Prince on Sunday night as the UN Security Council approves the deployment of an interim multinational force to the Caribbean country. \n ""An initial contingent of US troops arrived in Haiti just as the (UN Security) Council voted,"" US Ambassador John Negroponte to the United Nations said in New York. \n In Was...",20040229,"{'security': 1, 'council': 1}",{},"{'security': 2, 'council': 2, 'deploy': 3, 'force': 3, 'troop': 2, 'vote': 1, 'ambassador': 1, 'de

# convert to json, tar

In [8]:
df = pd.DataFrame(columns=['path','source', 'filename'])
df.path = [str(x) for x in pathlib.Path('423_filter/').glob('**/*csv')]

df.iloc[:,1:] = df.path.str.replace('423_filter/','').str.split('/', expand=True).values
df['date'] = df.path.apply(lambda x: re.search(r'_(\d\d\d\d\d\d)',x)[1]).values
df['year'] = df.date.apply(lambda x: x[0:4]).values.astype(int)
df.source = df.filename.apply(lambda x: re.search(r'(\w+_\w+)_\d',x)[1]).values
df['size'] = df['path'].parallel_apply(get_csv_size)
df

,path,source,filename,date,year,size
0,423_filter/xin_eng_200402.csv,xin_eng,xin_eng_200402.csv,200402,2004,7.6
1,423_filter/nyt_eng_199904.csv,nyt_eng,nyt_eng_199904.csv,199904,1999,18.9
2,423_filter/apw_eng_200904.csv,apw_eng,apw_eng_200904.csv,200904,2009,21.5
3,423_filter/ltw_eng_200407.csv,ltw_eng,ltw_eng_200407.csv,200407,2004,10.4
4,423_filter/xin_eng_200411.csv,xin_eng,xin_eng_200411.csv,200411,2004,7.2
...,...,...,...,...,...,...
874,423_filter/ltw_eng_199712.csv,ltw_eng,ltw_eng_199712.csv,199712,1997,6.3
875,423_filter/ltw_eng_199804.csv,ltw_eng,ltw_eng_199804.csv,199804,1998,6.6
876,423_filter/cna_eng_200805.csv,cna_eng,cna_eng_200805.csv,200805,2008,1.5
877,423_filter/xin_eng_200303.csv,xin_eng,xin_eng_200303.csv,200303,2003,10.9


In [9]:
print(df['size'].sum())
df.groupby(by=["source"], dropna=False)['size'].sum().round(2)

9666.300000000001


source
afp_eng    2664.9
apw_eng    3380.4
cna_eng     131.2
ltw_eng     950.3
nyt_eng    1273.3
wpb_eng      58.0
xin_eng    1208.2
Name: size, dtype: float64

In [10]:
df['json_file'] = df['filename'].apply(lambda x: x.replace('.csv', '.json'))
df['tar_file'] = '423_result/tar/'+ df.source+ '/'+df.json_file+'.tar.gz'
df.json_file = '423_result/json/'+ df.source+ '/'+df.json_file
df

,path,source,filename,date,year,size,json_file,tar_file
0,423_filter/xin_eng_200402.csv,xin_eng,xin_eng_200402.csv,200402,2004,7.6,423_result/json/xin_eng/xin_eng_200402.json,423_result/tar/xin_eng/xin_eng_200402.json.tar.gz
1,423_filter/nyt_eng_199904.csv,nyt_eng,nyt_eng_199904.csv,199904,1999,18.9,423_result/json/nyt_eng/nyt_eng_199904.json,423_result/tar/nyt_eng/nyt_eng_199904.json.tar.gz
2,423_filter/apw_eng_200904.csv,apw_eng,apw_eng_200904.csv,200904,2009,21.5,423_result/json/apw_eng/apw_eng_200904.json,423_result/tar/apw_eng/apw_eng_200904.json.tar.gz
3,423_filter/ltw_eng_200407.csv,ltw_eng,ltw_eng_200407.csv,200407,2004,10.4,423_result/json/ltw_eng/ltw_eng_200407.json,423_result/tar/ltw_eng/ltw_eng_200407.json.tar.gz
4,423_filter/xin_eng_200411.csv,xin_eng,xin_eng_200411.csv,200411,2004,7.2,423_result/json/xin_eng/xin_eng_200411.json,423_result/tar/xin_eng/xin_eng_200411.json.tar.gz
...,...,...,...,...,...,...,...,...
874,423_filter/ltw_eng_199712.csv,ltw_eng,ltw_eng_199712.csv,199712,1997,6.3,423_result/json/ltw_eng/ltw_eng_199712.json,423_result/tar/ltw_eng/ltw_eng_199712.json.tar.gz
875,423_filter/ltw_eng_199804.csv,ltw_eng,ltw_eng_199804.csv,199804,1998,6.6,423_result/json/ltw_eng/ltw_eng_199804.json,423_result/tar/ltw_eng/ltw_eng_199804.json.tar.gz
876,423_filter/cna_eng_200805.csv,cna_eng,cna_eng_200805.csv,200805,2008,1.5,423_result/json/cna_eng/cna_eng_200805.json,423_result/tar/cna_eng/cna_eng_200805.json.tar.gz
877,423_filter/xin_eng_200303.csv,xin_eng,xin_eng_200303.csv,200303,2003,10.9,423_result/json/xin_eng/xin_eng_200303.json,423_result/tar/xin_eng/xin_eng_200303.json.tar.gz


In [11]:
cwd = os.getcwd()
for i in df.source.unique():
    try:
        os.mkdir(cwd + '/423_result/json/'+i)
        os.mkdir(cwd + '/423_result/tar/'+i) 
    except:
        print('Already exist', end='\r')

In [12]:
df1 = pd.read_csv('423_filter/xin_eng_200402.csv', header=0)
df1

,id,title,text,date,head_in,head_out,text_in,text_out,head_in_count,head_out_count,body_in_count,body_out_count
0,XIN_ENG_20040201.0001,Sri Lankan govt says ready for cohabitation with president,"Sri Lankan government has said that it is ready to go ahead with what they called a ""structured co- habitation"" government with President Chandrika Kumaratunga which is likely to bring an end to the current political standoff in the country, The Sunday Times reported. \n Malik Samarawickrama, chairman of the ruling United National Party (UNP) of Prime Minister Ranil Wickremesinghe, said on Sat...",20040201,"{'president': 2, 'govern': 2}",{},"{'govern': 5, 'president': 7, 'political': 4, 'party': 1, 'minister': 2, 'committee': 3, 'council': 1, 'defense': 2, 'ministry': 2, 'ministries': 1, 'parliament': 1, 'election': 1}",{},2,0,12,0
1,XIN_ENG_20040201.0006,URGENT: Some 100 Iranian reformist MPs resign over election dispute,"Some 100 Iranian reformist MPs resigned Sunday to protest the mass disqualification of reformer candidates from the upcoming parliamentary elections, members of the Iranian parliament said.",20040201,"{'reform': 3, 'election': 2, 'dispute': 1, 'protest': 1, 'parliament': 1}",{},"{'reform': 2, 'protest': 1, 'parliament': 2, 'election': 1}",{},5,0,4,0
2,XIN_ENG_20040201.0007,Sri Lanka chief truce monitor assumes duty,"Chief truce monitor of the Nordic truce monitoring team in Sri Lanka supervising the ceasefire between the government and the Tamil Tiger rebels officially assumed his duty on Sunday as the new head of the group, officials said. \n Trond Furuhovde, a retired Norwegian army general, arrived in the island on Saturday morning, the spokesperson for the Sri Lanka Monitoring Mission (SLMM) said. \n ...",20040201,"{'govern': 1, 'rebel': 1}",{},"{'govern': 5, 'rebel': 6, 'officials': 1, 'army': 2, 'liberation': 1, 'armed': 1, 'war': 1, 'security': 1, 'defense': 1, 'minister': 1, 'political': 1}",{},2,0,11,0
3,XIN_ENG_20040201.0011,Karzai congratulates Afghans on Eid festival,"Afghanistan's transitional president Hamid Karzai on Sunday congratulated his people on the Eid-al-Adha festival, hoping peace and prosperity in the post-war country, which is still plagued by remnant of terrorism. \n ""I congratulate to the Afghan people on the occasion of Eid, and I wish all the luckiness and prosperity to our people and to the Islamic world,"" the president told reporters whi...",20040201,{'president': 1},{},"{'president': 5, 'war': 1, 'terror': 1, 'ambassador': 1, 'attack': 2, 'militia': 1, 'kill': 1, 'foreign': 1, 'peacekeep': 1, 'soldier': 1, 'bomb': 1, 'nato': 1, 'security': 1, 'force': 1}",{'dollar': 1},1,0,14,1
4,XIN_ENG_20040201.0014,"Bangladesh, Kuwait to sign defense pact","Bangladesh is going to sign a defense pact with Kuwait soon to provide technical and vocational training to Kuwaiti army. \n The Bangladeshi cabinet approved the Bangladesh-Kuwait Defense Pact on Saturday evening. \n A source from Bangladesh army elaborated the pact to Xinhua on Sunday that the pact will be in force for six years, and is renewable for another six years if the two sides agree. ...",20040201,"{'defense': 2, 'army': 1}",{},"{'defense': 4, 'army': 6, 'force': 2, 'armed': 2, 'forces': 1, 'peacekeep': 1, 'war': 1}","{'performance': 1, 'financ': 1, 'currency': 1}",2,0,7,3
...,...,...,...,...,...,...,...,...,...,...,...,...
4709,XIN_ENG_20040229.0278,US marines arrive in chaotic Haitian capital,"The first contingent of US Marines arrived in the chaotic Haitian capital of Port-au-Prince on Sunday night as the UN Security Council approves the deployment of an interim multinational force to the Caribbean country. \n ""An initial contingent of US troops arrived in Haiti just as the (UN Security) Council voted,"" US Ambassador John Negroponte to the United Nations said in New York. \n In Was...",20040229,"{'security': 1, 'council': 1}",{},"{'security': 2, 'council': 2, 'deploy': 3, 'force': 3, 'troop': 2, 'vote': 1, 'ambassador': 1, 'de

In [13]:
def convert_json_tar(filename, json_file, tar_file):
    df1 = pd.read_csv(filename, header=[0])
    df1 = df1[['title','text',	'date']]
            
    df1.to_json(json_file, orient="records", lines=True)
    
    with tarfile.open(tar_file, "w:gz") as tar:
        tar.add(json_file, arcname=os.path.basename(json_file))

pandarallel.initialize(nb_workers=38, progress_bar=True) 
_ = df.parallel_apply(lambda x: convert_json_tar(x['path'], x['json_file'], x['tar_file']), axis=1)

INFO: Pandarallel will run on 38 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [14]:
pandarallel.initialize() 

df = pd.DataFrame(columns=['path','source', 'filename'])
df.path = [str(x) for x in pathlib.Path('423_result/json/').glob('**/*json')]

# df.iloc[:,1:] = df.path.str.replace('417_filter/','').str.split('/', expand=True).values
# df['date'] = df.path.apply(lambda x: re.search(r'_(\d\d\d\d\d\d)',x)[1]).values
# df['year'] = df.date.apply(lambda x: x[0:4]).values.astype(int)
df.source = df.path.apply(lambda x: x.split('/')[2]).values
df['size'] = df['path'].parallel_apply(get_csv_size)
df

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


,path,source,filename,size
0,423_result/json/cna_eng/cna_eng_200802.json,cna_eng,NaN,0.8
1,423_result/json/cna_eng/cna_eng_200603.json,cna_eng,NaN,1.1
2,423_result/json/cna_eng/cna_eng_200311.json,cna_eng,NaN,0.6
3,423_result/json/cna_eng/cna_eng_200712.json,cna_eng,NaN,1.0
4,423_result/json/cna_eng/cna_eng_200506.json,cna_eng,NaN,1.1
...,...,...,...,...
874,423_result/json/nyt_eng/nyt_eng_199410.json,nyt_eng,NaN,11.7
875,423_result/json/nyt_eng/nyt_eng_199609.json,nyt_eng,NaN,17.4
876,423_result/json/nyt_eng/nyt_eng_199811.json,nyt_eng,NaN,15.3
877,423_result/json/nyt_eng/nyt_eng_199701.json,nyt_eng,NaN,16.8


In [15]:
print(df['size'].sum())
df.groupby(by=["source"], dropna=False)['size'].sum().round(1)

8818.2


source
afp_eng    2399.6
apw_eng    3074.5
cna_eng     119.6
ltw_eng     890.2
nyt_eng    1200.2
wpb_eng      54.3
xin_eng    1079.8
Name: size, dtype: float64